In [41]:
import hyper_param as hp
import evaluate
import numpy as np
import operator
import collections
import random
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = 20
#logging.info("")

In [42]:
#Define constants
tau = 10 #price history window
t_0 = 1000 #start point in history
num_dyna_updates = 25
ntrials = 10000
epsilon = 0.2
horizon = 100
gamma = 0.9

In [43]:
price = hp.get_price_history()

In [44]:
len(price)

2593

In [46]:
'''
Returns r, sp
'''
def sim(t, s, a, horizon_t):
    x = [t+i for i in range(tau)]
    y = price[t-tau:t]
    slope, intercept = np.polyfit(x, y, 1)
    r = 0
    ncoin = s[1]
    if ncoin <= -50 or ncoin >= 50 or t == horizon_t - 1:
        return ((price[t]*ncoin)[0], (hp.get_interval_enum(slope[0]), 0))
    else:
        if a == -1: # buy
            prob = random.uniform(0, 1)
            if prob <= 0.10:
                r = 0
            else:
                r = 0-price[t][0]
                ncoin += 1
        elif a == 1: # sell
            r = price[t][0]
            ncoin -= 1
        elif a == 0:
            reward = 0
        else:
            #throw expection
            print("Invalid action")
        sp = (hp.get_interval_enum(slope[0]), ncoin)
        return r, sp

In [47]:
'''
initialize N's, p, Q
'''
N_sasp = {} # key = N_sasp[(s, a)][sp] #value = transition count
N_sa = collections.defaultdict(int) #key = (s, a) #value = occurence count
p = collections.defaultdict(int) #key = (s, a) #value = sum of rewards
Q = {} # Q[s][a] = q value of taking action a in state s

In [48]:
'''
Returns chosen action
'''
def choose_action_egreedy(s, Q):
    if s in Q:
        Q_value = Q[s]
        prob = random.uniform(0, 1)
        if prob <= 1 - epsilon:  #exploit
            action = max(Q_value, key=Q_value.get)
            return action
        else:  #explore
            action = random.choice([-1, 0, 1])
            if action in Q_value:
                return action
            else:
                return action
    else:
        action = random.choice([-1, 0, 1])
        return action

In [49]:
def get_new_random_state_action(Q, updated):
    for i in range(5):
        rand_s = random.choice(Q.keys())
        rand_a = random.choice(Q[rand_s].keys())
        if (rand_s, rand_a) not in updated:
            return rand_s, rand_a
    return None, None

In [50]:
def dyna(Q, s, a, N_sa, N_sasp, p):
    R_sa = p[(s, a)]/ N_sa[(s, a)]
    
    summation = 0
    for sp in N_sasp[(s, a)]:
        T_sasp = N_sasp[(s, a)][sp]/N_sa[(s, a)]
        #compute max_a'[Q(s',a')]
        if sp not in Q:
            continue #Q = 0
        max_Q = max(Q[sp].iteritems(), key=operator.itemgetter(1))[1]
        summation += T_sasp*max_Q
    
    if s not in Q:
        Q[s] = {}
    if a not in Q[s]:
        Q[s][a] = 0
        
    Q[s][a] = R_sa + gamma*summation

In [51]:
def mle_based_rl(t, s_0):
    s = s_0   #initial state (ncoins, slope_interval)
    curr_horizon = t + horizon
    while (t < curr_horizon):
        
        a = choose_action_egreedy(s, Q)
        r, sp = sim(t, s, a, curr_horizon)
            
        #update N counts
        if (s , a) not in N_sa:
            N_sasp[(s, a)] = collections.defaultdict(int)
            
        N_sa[(s, a)] += 1
        N_sasp[(s, a)][sp] += 1
        
        #update p
        p[(s, a)] += r
        
        dyna(Q, s, a, N_sa, N_sasp, p)
        
        #update Q using Dyna strategy
        updated = set((s,a))
        for i in range(num_dyna_updates):
            s_rand, a_rand = get_new_random_state_action(Q, updated)
            if s_rand is None:   # nothing to update
                break
            dyna(Q, s_rand, a_rand, N_sa, N_sasp, p)
            updated.add((s_rand, a_rand))
        
        s = sp
        t += 1

In [ ]:
for i in range(ntrials):
    t = random.randint(t_0, len(price)-100)
    s_0 = (0, 0)
    if t > t_0:
        x = [t+i for i in range(tau)]
        y = price[t-tau:t]
        slope, intercept = np.polyfit(x, y, 1)
        s_0 = (hp.get_interval_enum(slope[0]), 0)
    
    #logging.info("MLE(t=%d, s_0=[%d, %d])", t, s_0[0], s_0[1])
    mle_based_rl(t, s_0)

In [37]:
print(Q)

with open('./mle_log.txt', 'w') as f:
     f.write(str(Q))


{(12, -43): {0: array([ 578.58229885]), 1: array([ 664.09586207]), -1: array([ 599.12155])}, (7, 3): {0: array([-600.942]), 1: array([-33.69])}, (18, -29): {0: array([ 1492.688]), 1: array([ 1987.3015748]), -1: array([ 1921.595])}, (11, -34): {0: array([ 357.33782051]), 1: array([ 395.18941606]), -1: array([ 236.952625])}, (16, 9): {-1: array([-863.054])}, (7, -37): {0: array([ 2012.52107692]), 1: array([ 2334.57421384]), -1: array([ 2544.95487923])}, (17, -32): {0: array([ 2098.06]), 1: array([ 3446.14902439]), -1: array([ 2540.95507042])}, (10, -45): {0: array([ 185.54949495]), 1: array([ 201.93013544]), -1: array([ 166.00485308])}, (4, -50): {0: array([ 59.326]), -1: array([ 0.])}, (6, -34): {0: array([ 1862.56857143]), 1: array([ 2099.71447368]), -1: array([ 1903.79113208])}, (16, -31): {0: array([ 1502.21105263]), 1: array([ 1125.12]), -1: array([ 1299.1796])}, (18, -3): {0: array([ 199.06]), 1: array([ 214.73263158]), -1: array([ 116.605])}, (17, -45): {0: array([ 1740.91111111])

In [15]:
Q = {(12, -43): {0: 0.0, 1: 21.497277108433714, -1: -24.730810810810809}, (7, 3): {1: 92.819999999999993}, (18, -29): {0: 0.0, 1: 59.575714285714277, -1: -44.953529411764713}, (11, -34): {0: 0.0, 1: 5.9832906666666741, -1: -5.2815151515151495}, (7, -37): {0: 0.0, 1: 71.485813953488417, -1: -42.843636363636364}, (17, -32): {0: 0.0, 1: 68.58629629629624, -1: -67.387027027027031}, (19, 4): {-1: -111.27500000000001}, (4, -50): {0: 2644.1666666666665, 1: 3088.0175438596493, -1: 2633.5}, (6, -34): {0: 0.0, 1: 63.670161290322575, -1: -38.403999999999996}, (16, -31): {0: 0.0, 1: 44.581610738255073, -1: -35.353500000000004}, (18, -3): {0: 0.0, 1: 57.502346368715067, -1: -35.68090909090909}, (17, -45): {0: 0.0, 1: 66.046823529411739, -1: -59.280000000000001}, (20, -23): {0: 0.0, 1: 135.6565486194483, -1: -136.59776000000002}, (5, -43): {0: 0.0, 1: 89.563763440860214, -1: -90.532499999999999}, (7, -15): {0: 0.0, 1: 68.780720338982945, -1: -66.062187500000022}, (17, -6): {0: 0.0, 1: 67.872140672782834, -1: -45.490645161290331}, (2, -40): {0: 0.0, 1: 55.315942028985503, -1: -54.470000000000013}, (19, -22): {0: 0.0, 1: 75.302611464968138, -1: -81.22230769230768}, (1, -17): {0: 0.0, 1: 132.54176795580136, -1: -119.28588235294117}, (8, 5): {-1: 0.0}, (6, -16): {0: 0.0, 1: 64.618918918918936, -1: -73.766000000000005}, (1, -33): {0: 0.0, 1: 134.76149253731339, -1: -142.41}, (3, -17): {0: 0.0, 1: 62.460186915887874, -1: -45.103333333333332}, (9, 0): {0: 0.0, 1: 27.640597014925383, -1: -28.823461538461537}, (3, -47): {0: 0.0, 1: 46.933124999999997, -1: -37.275833333333331}, (0, -38): {0: 0.0, 1: 132.10188727583284, -1: -122.80734939759036}, (2, -14): {0: 0.0, 1: 55.234455958549248, -1: -50.093499999999992}, (14, -10): {0: 0.0, 1: 35.290400000000005, -1: -24.218}, (4, -14): {0: 0.0, 1: 58.406510067114176, -1: -50.606249999999996}, (9, -50): {0: 1630.1443965517242, 1: 1099.0, -1: 1492.1923076923076}, (1, -15): {0: 0.0, 1: 132.155867346939, -1: -142.60999999999999}, (3, -11): {0: 0.0, 1: 52.912021276595738, -1: -56.016666666666673}, (14, 1): {1: 44.937857142857141, -1: -43.230000000000004}, (0, -16): {0: 0.0, 1: 128.59697374938122, -1: -120.65739726027394}, (2, -20): {0: 0.0, 1: 55.309017341040445, -1: -43.844000000000008}, (13, -13): {0: 0.0, 1: 28.723363636363636, -1: -23.750000000000004}, (15, 4): {-1: -69.510000000000005}, (15, -13): {0: 0.0, 1: 57.661486988847521, -1: -43.800555555555569}, (5, -35): {0: 0.0, 1: 101.62395833333335, -1: -110.4675}, (1, -21): {0: 0.0, 1: 135.56974842767301, -1: -84.034500000000008}, (12, -10): {0: 0.0, 1: 19.606779874213821, -1: -21.569264705882354}, (0, -22): {0: 0.0, 1: 130.29492807671858, -1: -113.77857142857134}, (9, -19): {0: 0.0, 1: 32.349399656946858, -1: -27.177058823529421}, (3, 2): {0: 0.0, -1: -27.969999999999999}, (11, -15): {0: 0.0, 1: 5.7187045965927155, -1: -5.8737259100642403}, (8, -20): {0: 0.0, 1: 52.863079847908686, -1: -43.429375}, (4, 5): {0: 0.0}, (10, -16): {0: 0.0, 1: 6.3519864226682428, -1: -5.6474144486692026}, (18, -38): {0: 0.0, 1: 59.791553398058227, -1: -64.673333333333332}, (19, -46): {0: 0.0, 1: 73.24045454545454, -1: -50.556666666666672}, (20, -1): {0: 0.0, 1: 128.53015521064356, -1: -115.24849673202617}, (11, -41): {0: 0.0, 1: 6.0255837563451848, -1: -5.1654399999999976}, (16, 0): {0: 0.0, 1: 38.408238636363635, -1: -23.618636363636366}, (7, -48): {0: 0.0, 1: 66.928903225806437, -1: -51.524000000000001}, (8, -14): {0: 0.0, 1: 63.710883392226073, -1: -42.973846153846154}, (10, -38): {0: 0.0, 1: 6.1910706214689357, -1: -5.9017712177121791}, (6, -47): {0: 0.0, 1: 58.95465116279069, -1: -65.371428571428581}, (16, -40): {0: 0.0, 1: 46.807954545454542, -1: -52.391428571428591}, (9, -39): {0: 0.0, 1: 31.781980198019792, -1: -31.756250000000005}, (3, -50): {0: 2525.5, 1: 2790.0294117647059, -1: 2068.3000000000002}, (20, -16): {0: 0.0, 1: 130.89506522972266, -1: -110.18110294117642}, (5, -34): {0: 0.0, 1: 88.679139784946216, -1: -64.385000000000005}, (7, -18): {0: 0.0, 1: 64.758249158249058, -1: -62.298928571428576}, (17, -13): {0: 0.0, 1: 67.580200668896268, -1: -71.431000000000012}, (2, -45): {0: 0.0, 1: 62.057325581395339, -1: -58.983750000000001}, (19, -13): {0: 0.0, 1: 76.890604026845637, -1: -75.846785714285716}, (4, -45): {0: 0.0, 1: 60.338305084745762, -1: -49.308333333333337}, (8, 12): {1: 132.38336000000001}, (6, -21): {0: 0.0, 1: 66.539468085106364, -1: -50.960000000000001}, (16, -1): {0: 0.0, 1: 42.211445783132568, -1: -46.811000000000007}, (1, -48): {0: 0.0, 1: 140.28431578947362, -1: -170.9928571428571}, (3, -44): {0: 0.0, 1: 57.551346153846161, -1: -52.803999999999995}, (20, -34): {0: 0.0, 1: 128.47696913002827, -1: -115.95972602739728}, (5, -8): {0: 0.0, 1: 91.648874172185472, -1: -67.258571428571415}, (0, -47): {0: 0.0, 1: 129.64318660287105, -1: -105.47494623655908}, (19, -39): {0: 0.0, 1: 78.936410256410241, -1: -66.422142857142859}, (4, -7): {0: 0.0, 1: 57.907578947368464, -1: -41.45882352941176}, (15, -46): {0: 0.0, 1: 53.432033898305143, -1: -64.457999999999998}, (19, 1): {0: 0.0, 1: 69.483124999999987}, (3, -6): {0: 0.0, 1: 55.978061224489792, -1: -54.767777777777781}, (14, -41): {0: 0.0, 1: 32.853673469387736, -1: -35.45933333333334}, (14, 8): {0: 20.894805000000002}, (0, -9): {0: 0.0, 1: 132.08796660117892, -1: -122.95094936708865}, (12, 8): {1: 117.60495}, (2, -25): {0: 0.0, 1: 55.135140186915869, -1: -54.979130434782626}, (13, -20): {0: 0.0, 1: 28.186299559471362, -1: -29.89423076923077}, (15, -8): {0: 0.0, 1: 53.218961538461507, -1: -50.049473684210533}, (1, -20): {0: 0.0, 1: 137.24182926829278, -1: -109.349}, (0, -40): {0: 0.0, 1: 130.99051565377533, -1: -114.62847826086953}, (12, -19): {0: 0.0, 1: 20.004980443285529, -1: -14.476231884057974}, (0, 14): {1: 126.01499999999999, -1: -139.56}, (14, -7): {0: 0.0, 1: 34.230647058823507, -1: -39.382380952380949}, (0, -19): {0: 0.0, 1: 128.33044556962082, -1: -114.51588957055212}, (11, -10): {0: 0.0, 1: 5.3973815892997825, -1: -4.7565021459227443}, (8, -47): {0: 0.0, 1: 66.582312925170058, -1: -34.410714285714285}, (15, -26): {0: 0.0, 1: 52.221363636363627, -1: -36.015000000000001}, (18, -42): {0: 0.0, 1: 57.841868131868118, -1: -67.588571428571427}, (10, -5): {0: 0.0, 1: 6.5507209847596828, -1: -6.6242738589211658}, (14, -29): {0: 0.0, 1: 36.126346153846157, -1: -26.437999999999995}, (5, 1): {1: 104.51285714285713, -1: -51.286363636363632}, (9, -8): {0: 0.0, 1: 26.308647342995172, -1: -34.749600000000008}, (16, 7): {1: 35.505499999999998}, (8, -7): {0: 0.0, 1: 54.601423487544395, -1: -81.935789473684196}, (20, -28): {0: 0.0, 1: 132.9914723926384, -1: -110.37896825396822}, (16, -33): {0: 0.0, 1: 40.892142857142886, -1: -29.014545454545456}, (18, -17): {0: 0.0, 1: 57.07017751479286, -1: -55.300833333333337}, (18, 5): {1: 102.12742857142857}, (7, -25): {0: 0.0, 1: 61.794879999999935, -1: -59.124285714285712}, (17, -12): {0: 0.0, 1: 66.989382716049292, -1: -71.881071428571445}, (19, -8): {0: 0.0, 1: 72.475232558139524, -1: -51.10777777777777}, (4, -38): {0: 0.0, 1: 64.475000000000009, -1: -66.396666666666661}, (6, -30): {0: 0.0, 1: 61.893648648648664, -1: -51.297500000000014}, (16, -11): {0: 0.0, 1: 42.229555555555606, -1: -44.437142857142859}, (17, -35): {0: 0.0, 1: 69.438597560975552, -1: -58.100909090909084}, (20, -27): {0: 0.0, 1: 132.62860025220749, -1: -107.71375939849621}, (5, -31): {0: 0.0, 1: 89.418373983739855, -1: -112.12666666666667}, (7, -3): {0: 0.0, 1: 67.417231404958571, -1: -44.346315789473678}, (19, -34): {0: 0.0, 1: 72.160357142857151, -1: -68.545384615384606}, (4, -32): {0: 0.0, 1: 58.915061728395081, -1: -68.033846153846156}, (15, -37): {0: 0.0, 1: 50.045181818181824, -1: -53.084545454545449}, (11, 4): {0: -50.823, 1: 19.286352941176474}, (3, -29): {0: 0.0, 1: 64.998088235294134, -1: -71.458571428571432}, (14, -34): {0: 0.0, 1: 34.781749999999995, -1: -31.548666666666669}, (0, -50): {0: 6161.2615384615383, 1: 6476.0368550368548, -1: 5791.1000000000004}, (2, -2): {0: 0.0, 1: 58.637962962962966, -1: -45.470999999999989}, (13, -27): {0: 0.0, 1: 28.996816143497739, -1: -15.609285714285717}, (13, -2): {0: 0.0, 1: 32.865343137254904, -1: -32.560000000000002}, (7, -47): {0: 0.0, 1: 77.433416666666659, -1: -91.404000000000011}, (12, -28): {0: 0.0, 1: 19.386240249609994, -1: -16.070232558139534}, (0, 5): {1: 172.47200000000001}, (0, -28): {0: 0.0, 1: 133.32014810045104, -1: -119.69852173913046}, (18, 0): {0: 0.0, 1: 61.597345679012314, -1: -45.683499999999995}, (11, -17): {0: 0.0, 1: 6.0142756410256251, -1: -4.5767967145790545}, (1, 0): {0: 0.0, 1: 136.69287581699362, -1: -93.774375000000006}, (15, -17): {0: 0.0, 1: 61.514151785714219, -1: -46.525652173913038}, (10, -30): {0: 0.0, 1: 6.4118601798855135, -1: -6.7781329923273717}, (2, 7): {-1: -171.39699999999999}, (19, -44): {0: 0.0, 1: 69.198888888888845, -1: -63.865000000000009}, (14, -22): {0: 0.0, 1: 35.312465116279064, -1: -26.595999999999997}, (5, 10): {0: 0.0}, (9, -31): {0: 0.0, 1: 32.08992207792209, -1: -31.523939393939393}, (18, -16): {0: 0.0, 1: 57.63229166666661, -1: -52.365714285714283}, (13, -39): {0: 0.0, 1: 33.365727272727284, -1: -31.584444444444443}, (6, 1): {1: 77.148571428571429}, (8, -32): {0: 0.0, 1: 65.963811881188036, -1: -60.665294117647058}, (10, -43): {0: 0.0, 1: 6.8505956944853956, -1: -5.2875581395348794}, (12, -40): {0: 0.0, 1: 16.863936430317832, -1: -11.6015625}, (7, 4): {1: 76.060000000000002}, (16, -42): {0: 0.0, 1: 42.513544303797467, -1: -42.759}, (18, -26): {0: 0.0, 1: 58.535129870129872, -1: -59.518888888888895}, (11, -37): {0: 0.0, 1: 5.6357168949771506, -1: -6.0904273504273467}, (17, -46): {0: 0.0, 1: 62.928579545454525, -1: -41.789999999999999}, (9, -42): {0: 0.0, 1: 30.612348484848461, -1: -45.790000000000006}, (17, -19): {0: 0.0, 1: 68.819653979238709, -1: -61.785555555555582}, (10, -42): {0: 0.0, 1: 6.85127094165221, -1: -5.5039781021897776}, (6, -27): {0: 0.0, 1: 66.288375000000002, -1: -65.156000000000006}, (16, -20): {0: 0.0, 1: 43.42376344086027, -1: -38.434285714285714}, (20, -20): {0: 0.0, 1: 134.10937463126913, -1: -103.83931972789115}, (5, -22): {0: 0.0, 1: 84.988370370370376, -1: -78.319375000000008}, (7, -6): {0: 0.0, 1: 71.974351145038085, -1: -56.449999999999996}, (2, -33): {0: 0.0, 1: 55.126831683168284, -1: -45.782000000000004}, (19, -25): {0: 0.0, 1: 69.407847222222216, -1: -54.532352941176462}, (4, -17): {0: 0.0, 1: 58.078833333333385, -1: -50.108333333333341}, (8, 0): {0: 0.0, 1: 56.862896341463355, -1: -60.054210526315785}, (12, 6): {0: 0.0}, (3, -24): {0: 0.0, 1: 61.07632653061227, -1: -45.261818181818185}, (9, -46): {0: 0.0, 1: 28.282448979591834, -1: -25.728518518518527}, (5, -12): {0: 0.0, 1: 87.166451612903217, -1: -109.18374999999997}, (10, -2): {0: 0.0, 1: 6.3833245072079929, -1: -4.9824070796460136}, (2, -7): {0: 0.0, 1: 52.823392857142835, -1: -51.274666666666675}, (4, -11): {0: 0.0, 1: 57.690496453900792, -1: -44.202999999999996}, (18, -10): {0: 0.0, 1: 59.181428571428526, -1: -53.273571428571429}, (15, -50): {0: 3435.9384615384615, 1: 3211.8529411764707, -1: 3295.3333333333335}, (1, -2): {0: 0.0, 1: 142.01954545454572, -1: -122.62111111111111}, (12, -29): {0: 0.0, 1: 19.79329073482424, -1: -19.074000000000002}, (0, -5): {0: 0.0, 1: 135.07659563924724, -1: -124.29974999999995}, (11, -28): {0: 0.0, 1: 5.8797944664031636, -1: -3.9670731707317075}, (13, -8): {0: 0.0, 1: 29.354669603524215, -1: -31.604444444444443}, (15, -12): {0: 0.0, 1: 55.368306451612845, -1: -49.7446153846154}, (10, -19): {0: 0.0, 1: 6.7188381804623338, -1: -6.3391976516633992}, (12, -7): {0: 0.0, 1: 18.729816401468774, -1: -16.806140350877197}, (14, -19): {0: 0.0, 1: 35.208689320388345, -1: -26.451052631578943}, (9, -22): {0: 0.0, 1: 31.906964912280756, -1: -30.333265306122453}, (17, -43): {0: 0.0, 1: 68.819312169312127, -1: -46.321111111111115}, (11, -6): {0: 0.0, 1: 5.5501320339515887, -1: -4.1025403225806443}, (13, -46): {0: 0.0, 1: 29.574830508474605, -1: -22.379166666666666}, (8, -25): {0: 0.0, 1: 57.43985185185177, -1: -59.685555555555553}, (18, -40): {0: 0.0, 1: 59.462795698924722, -1: -51.831666666666671}, (10, -9): {0: 0.0, 1: 6.3916613091758956, -1: -5.7258178438661709}, (12, -41): {0: 0.0, 1: 19.671520737327178, -1: -20.885769230769235}, (18, -31): {0: 0.0, 1: 58.243750000000006, -1: -39.789090909090909}, (11, -48): {0: 0.0, 1: 5.8991289522628474, -1: -3.7587822878228785}, (16, 11): {1: 89.102987404450005}, (20, -41): {0: 0.0, 1: 125.8064009111619, -1: -124.09590909090907}, (17, -26): {0: 0.0, 1: 65.828204081632592, -1: -46.504999999999995}, (19, 6): {-1: -53.705000000000005}, (9, -44): {0: 0.0, 1: 31.578272058823501, -1: -19.386666666666667}, (9, -38): {0: 0.0, 1: 30.568039867109622, -1: -32.070000000000007}, (16, -29): {0: 0.0, 1: 42.808802816901434, -1: -36.357777777777784}, (18, -5): {0: 0.0, 1: 54.813000000000017, -1: -52.368333333333325}, (20, -21): {0: 0.0, 1: 135.63073341094355, -1: -128.19775193798449}, (5, -45): {0: 0.0, 1: 95.2136, -1: -98.689999999999984}, (7, -13): {0: 0.0, 1: 68.862904411764589, -1: -39.483333333333327}, (17, -8): {0: 0.0, 1: 63.669039735099261, -1: -63.9438888888889}, (2, -42): {0: 0.0, 1: 55.556516853932564, -1: -40.501249999999999}, (19, -20): {0: 0.0, 1: 77.94528409090907, -1: -73.814999999999998}, (4, -42): {0: 0.0, 1: 59.17391304347823, -1: -51.696666666666665}, (8, 7): {-1: -87.420000000000002}, (6, -10): {0: 0.0, 1: 57.541192660550472, -1: -85.211666666666673}, (16, -7): {0: 0.0, 1: 42.017260273972653, -1: -38.71076923076923}, (1, -35): {0: 0.0, 1: 122.75177083333325, -1: -92.905833333333348}, (17, -33): {0: 0.0, 1: 64.773378995433731, -1: -55.597272727272724}, (20, -47): {0: 0.0, 1: 129.71234527687312, -1: -126.2103703703704}, (5, -3): {0: 0.0, 1: 89.613924731182863, -1: -97.216111111111104}, (0, -36): {0: 0.0, 1: 128.88734104046273, -1: -117.65388888888886}, (2, -16): {0: 0.0, 1: 57.339161290322593, -1: -58.397272727272728}, (10, 5): {0: 0.0, 1: 5.626666666666666, -1: 4.448599999999999}, (4, -4): {0: 0.0, 1: 56.580909090909131, -1: -62.531111111111109}, (15, -41): {0: 0.0, 1: 57.575317460317493, -1: -49.356923076923074}, (1, -9): {0: 0.0, 1: 128.02978494623665, -1: -103.11833333333333}, (11, 8): {1: 25.844999999999999}, (3, -9): {0: 0.0, 1: 53.449411764705872, -1: -45.907142857142858}, (14, -46): {0: 0.0, 1: 35.810973451327428, -1: -31.197142857142854}, (14, 7): {0: -2.6931869999999991}, (0, -14): {0: 0.0, 1: 129.69267224409501, -1: -120.3178181818182}, (2, -22): {0: 0.0, 1: 62.942643678160941, -1: -61.534705882352938}, (13, -15): {0: 0.0, 1: 27.433990384615399, -1: -31.363478260869559}, (15, 6): {1: 92.045714285714297}, (15, -3): {0: 0.0, 1: 61.135269230769147, -1: -72.247619047619054}, (1, -23): {0: 0.0, 1: 134.09833333333327, -1: -122.99312499999999}, (12, -16): {0: 0.0, 1: 20.343068181818229, -1: -17.623194444444444}, (0, 9): {0: 71.649000000000001, 1: 79.609999999999999}, (14, -12): {0: 0.0, 1: 35.386553672316374, -1: -45.576249999999995}, (0, -24): {0: 0.0, 1: 129.82964971751451, -1: -110.30719424460422}, (11, -13): {0: 0.0, 1: 5.4998806907378643, -1: -4.425646817248456}, (16, -38): {0: 0.0, 1: 34.990227272727253, -1: -33.856666666666676}, (8, -18): {0: 0.0, 1: 56.607455830388638, -1: -59.290499999999994}, (10, -1): {0: 0.0, 1: 6.4557015011547332, -1: -5.2544241316270544}, (20, 8): {-1: 69.697299999999998}, (12, -50): {0: 1005.2222222222222, 1: 843.22448979591832, -1: 1234.8934426229507}, (9, -11): {0: 0.0, 1: 27.242774674115442, -1: -38.223863636363632}, (9, -33): {0: 0.0, 1: 30.105738161559916, -1: -23.992000000000001}, (16, 2): {0: 0.0, 1: 34.892000000000003}, (7, -46): {0: 0.0, 1: 71.406350364963558, -1: -36.899000000000001}, (8, -12): {0: 0.0, 1: 61.648133333333234, -1: -45.359047619047622}, (10, -40): {0: 0.0, 1: 6.8888108561835439, -1: -5.7250337837837844}, (17, 3): {-1: -67.963999999999999}, (6, -41): {0: 0.0, 1: 70.239649122807023, -1: -93.075999999999993}, (3, -1): {0: 0.0, 1: 55.282750000000021, -1: -87.112500000000011}, (18, 6): {1: 116.37368571428571}, (18, -14): {0: 0.0, 1: 58.819390243902433, -1: -40.62466666666667}, (20, -14): {0: 0.0, 1: 127.509271817677, -1: -117.11843137254898}, (5, -36): {0: 0.0, 1: 90.212987012987, -1: -88.090000000000018}, (7, -24): {0: 0.0, 1: 60.445905511810878, -1: -38.299999999999997}, (17, -15): {0: 0.0, 1: 66.132973856209077, -1: -71.595600000000005}, (1, -40): {0: 0.0, 1: 143.53631578947369, -1: -121.52499999999999}, (19, -11): {0: 0.0, 1: 76.650683229813737, -1: -45.058749999999996}, (4, -35): {0: 0.0, 1: 55.344939759036144, -1: -44.651249999999997}, (6, -23): {0: 0.0, 1: 66.168055555555597, -1: -47.925555555555555}, (16, -16): {0: 0.0, 1: 42.899225806451661, -1: -25.203714285714288}, (1, -42): {0: 0.0, 1: 143.40499999999997, -1: -139.27687499999999}, (3, -42): {0: 0.0, 1: 62.796206896551723, -1: -57.94142857142856}, (20, -40): {0: 0.0, 1: 127.60942286348498, -1: -115.47382352941179}, (5, -26): {0: 0.0, 1: 88.045307692307716, -1: -77.305909090909097}, (0, -45): {0: 0.0, 1: 128.09747010119602, -1: -125.1505405405405}, (10, 12): {0: 0.0}, (4, -5): {0: 0.0, 1: 60.813695652174026, -1: -42.283333333333339}, (8, -38): {0: 0.0, 1: 58.533768115942046, -1: -82.968750000000014}, (15, -36): {0: 0.0, 1: 48.278173076923068, -1: -57.976363636363637}, (18, -49): {0: 0.0, 1: 63.029560439560449, -1: -45.289444444444449}, (13, -47): {0: 0.0, 1: 28.158809523809548, -1: -26.771250000000002}, (11, 1): {0: 0.0, 1: 5.6082151029748255, -1: -11.396571428571429}, (3, -4): {0: 0.0, 1: 63.754090909090927, -1: -57.873333333333335}, (2, -47): {0: 0.0, 1: 54.149558823529432, -1: -47.838571428571427}, (14, -43): {0: 0.0, 1: 39.130405405405412, -1: -27.177692307692304}, (18, -48): {0: 0.0, 1: 60.631470588235288, -1: -67.982857142857142}, (2, -27): {0: 0.0, 1: 56.719270833333319, -1: -48.277272727272731}, (13, -22): {0: 0.0, 1: 32.590054644808717, -1: -24.296666666666667}, (15, -6): {0: 0.0, 1: 60.097471264367734, -1: -43.866874999999993}, (13, 11): {0: 7.9704000000000077}, (1, -30): {0: 0.0, 1: 135.76435483870961, -1: -122.16999999999999}, (12, -17): {0: 0.0, 1: 20.533470437017993, -1: -15.88490566037736}, (0, 0): {0: 0.0, 1: 134.23370976253332, -1: -125.38917241379308}, (0, -17): {0: 0.0, 1: 128.70734951456359, -1: -118.62512820512818}, (11, -24): {0: 0.0, 1: 6.2621368272951035, -1: -6.5809192825112133}, (15, -32): {0: 0.0, 1: 58.047734806629819, -1: -47.374000000000002}, (10, -7): {0: 0.0, 1: 6.6467927823050115, -1: -5.375383141762458}, (9, 9): {0: 39.226499999999994}, (14, -31): {0: 0.0, 1: 33.983809523809533, -1: -33.47727272727272}, (9, -1): {0: 0.0, 1: 27.981943127962118, -1: -26.30040816326531}, (11, -50): {0: 317.07330415754922, 1: 324.45249887438092, -1: 256.17553191489361}, (0, -1): {0: 0.0, 1: 134.66767223382075, -1: -122.93862499999997}, (13, -34): {0: 0.0, 1: 30.095981308411215, -1: -28.416956521739131}, (8, -5): {0: 0.0, 1: 59.519900332225944, -1: -60.210869565217386}, (9, -9): {0: 0.0, 1: 26.235907473309585, -1: -18.607446808510641}, (1, -32): {0: 0.0, 1: 126.71676691729319, -1: -114.925}, (17, 4): {1: 93.734999999999985}, (7, 9): {-1: -24.289999999999999}, (16, -47): {0: 0.0, 1: 35.93540540540539, -1: -36.461764705882359}, (18, -19): {0: 0.0, 1: 58.324878048780455, -1: -45.145000000000003}, (20, -7): {0: 0.0, 1: 126.15970443349805, -1: -112.99365217391298}, (7, -31): {0: 0.0, 1: 65.851041666666646, -1: -62.846666666666671}, (17, -22): {0: 0.0, 1: 70.412430555555545, -1: -64.285769230769247}, (19, -6): {0: 0.0, 1: 70.312173913043495, -1: -86.807272727272718}, (9, 4): {-1: -47.22428571428572}, (6, -32): {0: 0.0, 1: 64.851111111111123, -1: -72.143000000000001}, (16, -9): {0: 0.0, 1: 40.21213017751483, -1: -46.525625000000005}, (3, -33): {0: 0.0, 1: 55.234426229508209, -1: -28.576666666666664}, (20, -25): {0: 0.0, 1: 135.47542279411823, -1: -129.33040650406502}, (5, -17): {0: 0.0, 1: 91.400363636363664, -1: -89.226666666666674}, (19, -32): {0: 0.0, 1: 73.237232142857152, -1: -68.307894736842101}, (4, -30): {0: 0.0, 1: 59.44700934579442, -1: -69.748000000000019}, (6, -6): {0: 0.0, 1: 61.830267857142871, -1: -54.316249999999997}, (11, 6): {1: 5.3049999999999997}, (17, -39): {0: 0.0, 1: 72.252727272727242, -1: -63.842142857142861}, (14, -36): {0: 0.0, 1: 35.962685185185173, -1: -39.278333333333336}, (12, 1): {0: 0.0, 1: 26.739354838709676, -1: -23.374285714285715}, (2, -4): {0: 0.0, 1: 56.885657894736802, -1: -49.43333333333333}, (13, -29): {0: 0.0, 1: 30.299246231155774, -1: -17.991666666666667}, (6, -48): {0: 0.0, 1: 60.785937499999996, -1: -46.576999999999998}, (13, 4): {-1: -25.555}, (1, -5): {0: 0.0, 1: 125.32093264248716, -1: -77.965833333333336}, (12, -26): {0: 0.0, 1: 19.364318181818192, -1: -20.254193548387097}, (0, 7): {0: 0.0, -1: -116.90000000000001}, (19, -37): {0: 0.0, 1: 78.255688073394467, -1: -50.293999999999997}, (0, -26): {0: 0.0, 1: 131.00690504807753, -1: -119.08983870967738}, (11, -31): {0: 0.0, 1: 5.5784117915905034, -1: -5.5898123324396787}, (1, 6): {1: 232.19480000000001}, (13, -3): {0: 0.0, 1: 30.116620370370377, -1: -17.812999999999999}, (15, -23): {0: 0.0, 1: 50.531336206896512, -1: -53.277058823529408}, (10, -32): {0: 0.0, 1: 7.1677808407994421, -1: -4.7347747747747766}, (2, 5): {0: 47.654999999999994}, (12, -4): {0: 0.0, 1: 19.828296836982933, -1: -13.778026315789472}, (14, -24): {0: 0.0, 1: 37.188563218390804, -1: -32.902727272727276}, (5, 8): {-1: -82.730000000000004}, (9, -25): {0: 0.0, 1: 32.482859813084119, -1: -33.616511627906974}, (13, -41): {0: 0.0, 1: 30.154560000000025, -1: -15.7075}, (6, 7): {1: 286.26532000000003}, (8, -30): {0: 0.0, 1: 61.305687203791472, -1: -64.608260869565214}, (12, -38): {0: 0.0, 1: 18.466581395348825, -1: -24.176923076923078}, (7, 6): {-1: -129.56799999999998}, (18, -28): {0: 0.0, 1: 58.853353658536548, -1: -45.623124999999995}, (11, -35): {0: 0.0, 1: 5.4490739191073896, -1: -4.2854022988505767}, (5, -50): {0: 3803.0540540540542, 1: 4107.8999999999996, -1: 4312.3870967741932}, (18, -45): {0: 0.0, 1: 57.686666666666625, -1: -47.297000000000011}, (7, -34): {0: 0.0, 1: 70.632602739725996, -1: -55.164687500000021}, (17, -29): {0: 0.0, 1: 63.332509960159328, -1: -57.399999999999999}, (19, 3): {-1: -71.507499999999993}, (6, -37): {0: 52.7868947368421, 1: 66.440444444444438, -1: -63.277999999999999}, (16, -18): {0: 0.0, 1: 42.634022988505805, -1: -36.158571428571427}, (18, 10): {-1: -74.859999999999999}, (20, -18): {0: 0.0, 1: 130.28228963938207, -1: -113.32819548872178}, (5, -24): {0: 0.0, 1: 85.736465517241371, -1: -74.916666666666686}, (7, -12): {0: 0.0, 1: 72.637464285714174, -1: -46.355624999999996}, (2, -35): {0: 0.0, 1: 55.718333333333327, -1: -32.327999999999996}, (19, -23): {0: 0.0, 1: 80.432674418604734, -1: -56.352499999999992}, (4, -23): {0: 0.0, 1: 60.512063492063533, -1: -38.5978947368421}, (8, 2): {0: -39.204000000000001, 1: 48.179166666666667}, (6, -3): {0: 0.0, 1: 66.040740740740731, -1: -51.852499999999999}, (1, -38): {0: 0.0, 1: 129.02318181818174, -1: -117.21166666666666}, (3, -22): {0: 0.0, 1: 57.226712328767114, -1: -42.63111111111111}, (9, 3): {0: 19.4175, 1: 21.574999999999999}, (5, -14): {0: 0.0, 1: 86.676623376623354, -1: -93.935000000000002}, (18, 3): {0: 0.0}, (2, -9): {0: 0.0, 1: 60.550467836257347, -1: -41.616363636363637}, (19, -49): {0: 0.0, 1: 78.562808988764033, -1: -63.709999999999994}, (4, -9): {0: 0.0, 1: 59.460755813953583, -1: -59.76285714285715}, (8, -40): {0: 0.0, 1: 71.204166666666637, -1: -90.28666666666669}, (18, -47): {0: 0.0, 1: 64.826000000000008, -1: -37.462222222222216}, (1, -4): {0: 0.0, 1: 120.81825301204823, -1: -106.17200000000001}, (19, -43): {0: 0.0, 1: 74.283711340206168, -1: -80.079999999999984}, (3, -16): {0: 0.0, 1: 58.70207792207794, -1: -45.614999999999995}, (14, 2): {1: 25.772307692307688, -1: -7.8060000000000009}, (0, -3): {0: 0.0, 1: 135.23896394108735, -1: -111.42209790209785}, (11, -26): {0: 0.0, 1: 6.3307367260701817, -1: -4.5217730496453905}, (13, -10): {0: 0.0, 1: 29.545813008130057, -1: -23.051052631578951}, (15, -10): {0: 0.0, 1: 51.986119402985075, -1: -49.878333333333323}, (10, -21): {0: 0.0, 1: 7.2603310657596545, -1: -5.6863906581740986}, (9, -36): {0: 0.0, 1: 29.284093959731532, -1: -24.326153846153847}, (12, -5): {0: 0.0, 1: 21.104366197183126, -1: -16.320833333333333}, (14, -13): {0: 0.0, 1: 35.894062499999983, -1: -33.111000000000004}, (9, -24): {0: 0.0, 1: 30.917952029520315, -1: -25.56522727272727}, (3, 1): {0: 0.0, -1: -69.852500000000006}, (11, -4): {0: 0.0, 1: 5.7262564342536413, -1: -5.4859267734553798}, (9, -41): {0: 0.0, 1: 35.700735294117635, -1: -25.941499999999998}, (13, -48): {0: 0.0, 1: 28.373692307692295, -1: -20.654444444444447}, (8, -23): {0: 0.0, 1: 58.69178913738012, -1: -47.302380952380943}, (10, -11): {0: 0.0, 1: 6.6196041943583053, -1: -6.2774394463667829}, (6, 2): {1: 109.23541935483871}, (12, -47): {0: 0.0, 1: 20.737864583333323, -1: -13.595740740740737}, (16, -49): {0: 36.100894736842115, 1: 44.273571428571429, -1: -34.724333333333341}, (16, -2): {0: 0.0, 1: 43.138152173913106, -1: -37.13095238095238}, (9, -14): {0: 0.0, 1: 30.25994454713496, -1: -33.9221875}, (17, -42): {0: 0.0, 1: 68.240890052355965, -1: -59.049166666666672}, (11, -46): {0: 0.0, 1: 5.6637461773700144, -1: -4.3531854838709663}, (7, -41): {0: 0.0, 1: 72.846201550387619, -1: -68.747894736842113}, (17, -28): {0: 0.0, 1: 61.377451737451658, -1: -64.333199999999991}, (18, -36): {0: 0.0, 1: 56.580380952380942, -1: -80.985384615384618}, (10, -33): {0: 0.0, 1: 6.534666023631523, -1: -6.7879276315789463}, (6, -46): {0: 0.0, 1: 54.573863636363626, -1: -84.852500000000006}, (16, -27): {0: 0.0, 1: 39.313684210526354, -1: -41.395000000000003}, (20, 3): {0: 82.046250000000001, 1: 91.162499999999994, -1: -66.242500000000007}, (20, -11): {0: 0.0, 1: 127.7174709463204, -1: -111.1977857142857}, (5, -47): {0: 0.0, 1: 91.069279279279257, -1: -65.67647058823529}, (7, -19): {0: 0.0, 1: 70.107931034482675, -1: -58.074000000000005}, (17, -1): {0: 0.0, 1: 67.490581818181781, -1: -62.568750000000023}, (2, -44): {0: 0.0, 1: 58.148169014084516, -1: -38.098333333333336}, (19, -18): {0: 0.0, 1: 73.97904494382027, -1: -78.667999999999992}, (4, -48): {0: 0.0, 1: 56.853777777777779, -1: -52.873750000000001}, (6, -12): {0: 0.0, 1: 62.792880000000004, -1: -48.793749999999996}, (16, -5): {0: 0.0, 1: 41.349187500000042, -1: -21.639999999999997}, (1, -45): {0: 0.0, 1: 145.95791666666662, -1: -32.948}, (3, -45): {0: 0.0, 1: 47.535813953488379, -1: -50.04666666666666}, (20, -45): {0: 0.0, 1: 128.59956953642373, -1: -121.46950617283949}, (5, -5): {0: 0.0, 1: 93.727562500000033, -1: -100.56777777777778}, (0, -34): {0: 0.0, 1: 131.98915481832552, -1: -101.49092783505155}, (6, -1): {0: 0.0, 1: 64.018791208791214, -1: -48.663333333333334}, (10, 3): {0: 0.0, 1: 22.193076923076923, -1: -79.524999999999991}, (4, -1): {0: 0.0, 1: 57.828439716312147, -1: -42.474736842105266}, (15, -47): {0: 0.0, 1: 50.048549618320642, -1: -38.07769230769231}, (1, -11): {0: 0.0, 1: 135.6844311377246, -1: -111.36208333333333}, (3, -7): {0: 0.0, 1: 56.596438356164398, -1: -47.373076923076923}, (14, -48): {0: 0.0, 1: 32.80724409448819, -1: -24.463846153846152}, (14, 5): {1: -47.349000000000004}, (0, -12): {0: 0.0, 1: 131.65710284886575, -1: -116.01872483221472}, (2, -24): {0: 0.0, 1: 57.607272727272701, -1: -45.051249999999996}, (13, -17): {0: 0.0, 1: 25.579427312775334, -1: -27.338888888888889}, (15, 0): {0: 0.0, 1: 60.241223628691948, -1: -45.565806451612893}, (10, -36): {0: 0.0, 1: 6.5852472826086892, -1: -6.7468646864686477}, (12, -14): {0: 0.0, 1: 19.54134502923975, -1: -23.652812500000003}, (14, -6): {0: 0.0, 1: 33.201340782122891, -1: -21.029999999999998}, (18, 1): {0: 0.0, 1: 65.652499999999989, -1: -47.049999999999997}, (11, -11): {0: 0.0, 1: 5.7577210246797845, -1: -5.5111844660194178}, (20, -42): {0: 0.0, 1: 127.86857299670692, -1: -103.78925925925927}, (10, 7): {1: 12.788599999999999}, (15, -27): {0: 0.0, 1: 61.380833333333293, -1: -39.859999999999999}, (4, 1): {0: 0.0, 1: 54.855384615384608, -1: -50.859999999999999}, (10, -4): {0: 0.0, 1: 6.5547243052492252, -1: -6.2898192771084274}, (5, -11): {0: 0.0, 1: 86.616162162162169, -1: -91.354705882352931}, (5, 4): {0: 17.357717647058827, 1: 135.39499999999998}, (9, -5): {0: 0.0, 1: 26.645393634840865, -1: -33.210161290322581}, (2, -50): {0: 2582.0933333333332, 1: 2348.6428571428573, -1: 1797.0}, (16, 4): {1: 42.3185}, (8, -10): {0: 0.0, 1: 65.644825396825297, -1: -65.767391304347825}, (8, -48): {0: 0.0, 1: 70.35801526717556, -1: -72.491818181818175}, (4, -28): {0: 0.0, 1: 55.060232558139589, -1: -45.618333333333332}, (17, 1): {0: 62.534812499999987, 1: 79.262, -1: -49.109047619047629}, (6, -43): {0: 0.0, 1: 64.046842105263153, -1: -41.896000000000001}, (16, -36): {0: 0.0, 1: 36.62941176470585, -1: -20.921666666666667}, (20, 10): {-1: -131.97}, (20, -4): {0: 0.0, 1: 126.60147397260353, -1: -117.68804347826089}, (5, -38): {0: 0.0, 1: 78.929444444444442, -1: -84.87777777777778}, (7, -22): {0: 0.0, 1: 65.014826388888778, -1: -97.118076923076927}, (17, -9): {0: 0.0, 1: 67.981699999999918, -1: -71.811199999999999}, (19, -9): {0: 0.0, 1: 75.221698113207594, -1: -69.677499999999995}, (4, -33): {0: 0.0, 1: 57.046774193548409, -1: -75.86333333333333}, (6, -17): {0: 0.0, 1: 72.000833333333333, -1: -66.69285714285715}, (16, -14): {0: 0.0, 1: 42.204207920792115, -1: -45.023846153846158}, (1, -44): {0: 116.07047181818184, 1: 145.15289473684206, -1: -121.53142857142858}, (3, -40): {0: 0.0, 1: 48.538695652173921, -1: -21.432499999999997}, (20, -38): {0: 0.0, 1: 126.98950276243107, -1: -117.0652542372881}, (5, -28): {0: 0.0, 1: 90.387184466019406, -1: -72.337999999999994}, (0, -43): {0: 0.0, 1: 128.98217391304362, -1: -122.20408450704225}, (19, -35): {0: 0.0, 1: 67.483599999999981, -1: -74.185555555555553}, (4, -27): {0: 0.0, 1: 56.538347826087005, -1: -50.946153846153848}, (15, -34): {0: 0.0, 1: 57.199292035398258, -1: -34.377499999999998}, (1, -50): {0: 5599.3000000000002, 1: 6074.3684210526317, -1: 6692.598484848485}, (11, 3): {0: 0.0, 1: 16.857058823529414, -1: -4.8300000000000001}, (3, -2): {0: 0.0, 1: 54.218409090909105, -1: -31.667999999999996}, (14, -37): {0: 0.0, 1: 38.528000000000006, -1: -45.976666666666667}, (12, 4): {-1: -9.5150000000000006}, (2, -29): {0: 0.0, 1: 60.518837209302241, -1: -28.650833333333335}, (9, -40): {0: 0.0, 1: 33.923689655172431, -1: -25.401363636363637}, (13, -24): {0: 0.0, 1: 27.195707547169807, -1: -20.07}, (8, -42): {0: 0.0, 1: 67.476326530612241, -1: -105.94130434782613}, (9, -2): {0: 0.0, 1: 24.58634645669289, -1: -24.08979166666667}, (20, -30): {0: 0.0, 1: 132.85950116913514, -1: -124.19019230769224}, (12, -23): {0: 0.0, 1: 18.563196405648274, -1: -13.23066666666667}, (0, 2): {1: 105.67833333333334, -1: -90.245000000000005}, (14, -3): {0: 0.0, 1: 35.655450450450452, -1: -36.658749999999991}, (0, -31): {0: 0.0, 1: 133.82764536970581, -1: -124.79045871559634}, (11, -22): {0: 0.0, 1: 5.6830058577405751, -1: -6.2756136363636328}, (1, 3): {0: -54.149142857142863}, (15, -30): {0: 0.0, 1: 52.404491978609606, -1: -43.409047619047612}, (12, -30): {0: 0.0, 1: 16.96199335548172, -1: -20.915522388059699}, (10, -25): {0: 0.0, 1: 6.1590116949431586, -1: -6.2977890466531425}, (14, -25): {0: 0.0, 1: 37.340599078341029, -1: -37.971999999999994}, (9, -4): {0: 0.0, 1: 26.383642611683861, -1: -20.771111111111114}, (15, -43): {0: 0.0, 1: 55.774132231404998, -1: -56.093902439024397}, (13, -36): {0: 0.0, 1: 30.700317460317486, -1: -15.678333333333335}, (1, -39): {0: 0.0, 1: 120.46132530120481, -1: -133.63}, (8, -3): {0: 0.0, 1: 64.298527397260187, -1: -54.495789473684212}, (12, -35): {0: 0.0, 1: 16.187482352941171, -1: -16.413953488372094}, (16, -45): {0: 0.0, 1: 38.534476190476184, -1: -44.29666666666666}, (18, -21): {0: 0.0, 1: 58.759370629370615, -1: -43.529090909090911}, (17, -48): {0: 0.0, 1: 66.49157894736841, -1: -48.756428571428572}, (20, -5): {0: 0.0, 1: 126.98771863117925, -1: -109.48322147651004}, (7, -29): {0: 0.0, 1: 73.173060344827547, -1: -57.170909090909078}, (17, -24): {0: 0.0, 1: 66.081543624161071, -1: -67.664285714285697}, (18, -34): {0: 0.0, 1: 56.217931034482746, -1: -54.369230769230768}, (19, -4): {0: 0.0, 1: 71.311243243243226, -1: -81.677692307692297}, (6, -26): {0: 0.0, 1: 65.078815789473694, -1: -47.966363636363646}, (16, -23): {0: 0.0, 1: 44.091631205673821, -1: -36.527333333333338}, (20, -31): {0: 0.0, 1: 131.9980750000002, -1: -126.63383838383838}, (5, -19): {0: 0.0, 1: 85.24623456790124, -1: -69.386842105263156}, (7, -7): {0: 0.0, 1: 67.534854771784083, -1: -80.833684210526329}, (9, -49): {0: 0.0, 1: 29.093424124513586, -1: -27.387407407407409}, (19, -30): {0: 0.0, 1: 77.15568493150684, -1: -67.431428571428569}, (4, -20): {0: 0.0, 1: 56.345075757575792, -1: -51.696666666666658}, (6, -8): {0: 0.0, 1: 63.93010204081633, -1: -46.902222222222214}, (3, -25): {0: 0.0, 1: 49.555833333333325, -1: -31.224545454545453}, (20, -49): {0: 0.0, 1: 134.22049751243773, -1: -117.81266666666667}, (5, -9): {0: 0.0, 1: 94.200833333333449, -1: -79.913157894736869}, (12, 3): {0: 0.0, 1: -4.7407000000000004, -1: -4.3600000000000003}, (2, -6): {0: 0.0, 1: 56.723312499999984, -1: -49.772941176470582}, (13, -31): {0: 0.0, 1: 26.751769230769252, -1: -21.860769230769229}, (13, 2): {0: 0.0}, (1, -7): {0: 0.0, 1: 125.04263888888892, -1: -108.0425}, (12, -32): {0: 0.0, 1: 16.567632135306546, -1: -15.539636363636363}, (0, -8): {0: 0.0, 1: 133.19822596153867, -1: -110.58116279069765}, (11, -29): {0: 0.0, 1: 5.7672269084774515, -1: -4.4640263157894715}, (1, 4): {0: 0.0}, (13, -5): {0: 0.0, 1: 27.455095238095236, -1: -29.895000000000007}, (15, -21): {0: 0.0, 1: 62.085590551181035, -1: -50.677857142857135}, (10, -18): {0: 0.0, 1: 6.7375475655430446, -1: -6.1356083650190154}, (2, 3): {-1: -76.064999999999998}, (12, -1): {0: 0.0, 1: 19.372614213197931, -1: -18.624430379746833}, (14, -18): {0: 0.0, 1: 36.193317535545013, -1: -42.563750000000006}, (9, -27): {0: 0.0, 1: 32.837228464419525, -1: -31.046923076923068}, (19, 8): {-1: 51.827299999999994}, (11, -7): {0: 0.0, 1: 5.4822313792567225, -1: -5.9994674556213043}, (13, -43): {0: 0.0, 1: 26.735549132947991, -1: -22.760000000000002}, (6, 5): {0: 0.0}, (8, -28): {0: 0.0, 1: 53.853749999999934, -1: -55.121818181818178}, (12, -44): {0: 0.0, 1: 16.910840108401072, -1: -18.237352941176468}, (7, 0): {0: 0.0, 1: 73.327866108786552, -1: -65.622608695652175}, (18, -30): {0: 0.0, 1: 56.34659259259255, -1: -45.655000000000008}, (11, -33): {0: 0.0, 1: 5.9461817726589974, -1: -5.3788252148997113}, (7, -40): {0: 0.0, 1: 68.10368000000004, -1: -57.640625}, (17, -31): {0: 0.0, 1: 71.326666666666597, -1: -107.07363636363635}, (19, 5): {-1: -71.734499999999997}, (6, -39): {0: 0.0, 1: 49.676206896551705, -1: -58.929999999999993}, (16, -32): {0: 0.0, 1: 40.556503496503531, -1: -57.710000000000008}, (20, 6): {1: 138.58368571428571}, (6, -50): {1: 2203.1666666666665, -1: 3428.3194444444443}, (20, 2): {0: 82.456500000000005, 1: 91.618333333333339, -1: -64.375}, (5, -42): {0: 0.0, 1: 88.367977528089867, -1: -92.778461538461528}, (9, 7): {-1: -18.235500000000002}, (7, -10): {0: 0.0, 1: 69.085871212121091, -1: -45.805714285714281}, (17, -5): {0: 0.0, 1: 66.31099999999995, -1: -65.916315789473686}, (2, -37): {0: 0.0, 1: 54.326031746031752, -1: -69.556250000000006}, (19, -21): {0: 0.0, 1: 68.574647887323906, -1: -62.649000000000001}, (4, -21): {0: 0.0, 1: 57.81472000000003, -1: -39.363636363636367}, (8, 4): {-1: -56.469999999999999}, (6, -13): {0: 0.0, 1: 65.597222222222243, -1: -65.766428571428563}, (8, -43): {0: 0.0, 1: 71.128285714285695, -1: -58.569000000000003}, (3, -20): {0: 0.0, 1: 60.504642857142827, -1: -42.16846153846155}, (9, 1): {0: 0.0, 1: 23.088749999999997, -1: -29.266956521739136}, (5, -16): {0: 0.0, 1: 87.872392344497669, -1: -86.007222222222225}, (0, -39): {0: 0.0, 1: 130.31669058295995, -1: -130.08317757009345}, (2, -11): {0: 0.0, 1: 57.34209039548022, -1: -44.679999999999993}, (19, -47): {0: 0.0, 1: 79.117750000000015, -1: -68.512500000000003}, (4, -15): {0: 0.0, 1: 58.393391812865552, -1: -45.521818181818176}, (1, -14): {0: 0.0, 1: 136.16005494505504, -1: -84.468125000000001}, (3, -14): {0: 0.0, 1: 55.423235294117667, -1: -52.872499999999995}, (14, -49): {0: 0.0, 1: 40.112105263157908, -1: -33.111249999999998}, (14, 0): {0: 0.0, 1: 37.624867724867698, -1: -47.633333333333326}, (2, -17): {0: 0.0, 1: 53.92777777777772, -1: -62.06454545454546}, (19, -41): {0: 0.0, 1: 78.157094017093982, -1: -64.480000000000004}, (13, -12): {0: 0.0, 1: 28.200217391304353, -1: -31.78875}, (15, 5): {-1: -63.783333333333331}, (15, -16): {0: 0.0, 1: 55.312851239669371, -1: -43.802083333333321}, (18, -43): {0: 0.0, 1: 55.71467889908255, -1: -55.071428571428562}, (10, -23): {0: 0.0, 1: 6.4442992985191063, -1: -4.3658531746031777}, (12, -11): {0: 0.0, 1: 20.560313630880589, -1: -19.507794117647055}, (14, -15): {0: 0.0, 1: 34.161699999999975, -1: -31.226666666666663}, (9, -18): {0: 0.0, 1: 30.91685567010315, -1: -26.779166666666672}, (3, 3): {0: 179.06595000000002}, (11, -2): {0: 0.0, 1: 5.4933841368584835, -1: -4.3295218295218261}, (13, -50): {0: 1703.5530303030303, 1: 1542.625, -1: 1558.9239130434783}, (8, -21): {0: 0.0, 1: 59.95962068965509, -1: -54.748461538461541}, (16, -46): {0: 0.0, 1: 37.732500000000002, -1: -35.876153846153848}, (10, -13): {0: 0.0, 1: 6.519184798807748, -1: -5.3106692913385816}, (12, -45): {0: 0.0, 1: 17.912269326683276, -1: -23.554137931034475}, (18, -35): {0: 0.0, 1: 59.370588235294122, -1: -48.358666666666657}, (11, -44): {0: 0.0, 1: 5.8628844966028399, -1: -5.2278400000000005}, (15, 3): {1: 37.331666666666671}, (8, -15): {0: 0.0, 1: 63.968375451263505, -1: -80.08499999999998}, (10, -35): {0: 0.0, 1: 6.2530870712400981, -1: -6.5893771626297584}, (15, -2): {0: 0.0, 1: 57.076583629893179, -1: -39.091428571428573}, (16, -25): {0: 0.0, 1: 41.888689655172442, -1: -38.838157894736831}, (20, 13): {0: 140.30099999999999, 1: 155.88999999999999}, (3, -49): {0: 0.0, 1: 46.490232558139532, -1: -52.520000000000003}, (20, -9): {0: 0.0, 1: 125.84457283680224, -1: -115.80191176470595}, (5, -33): {0: 0.0, 1: 97.170277777777827, -1: -75.599999999999994}, (7, -17): {0: 0.0, 1: 64.452463235293976, -1: -54.60307692307692}, (17, -4): {0: 0.0, 1: 67.033930817609999, -1: -65.084799999999987}, (2, -46): {0: 0.0, 1: 59.769901960784303, -1: -67.810000000000002}, (19, -16): {0: 0.0, 1: 73.747142857142904, -1: -69.328823529411764}, (4, -46): {0: 0.0, 1: 58.383975903614449, -1: -47.89800000000001}, (6, -22): {0: 0.0, 1: 63.98312500000003, -1: -55.740666666666655}, (16, -3): {0: 0.0, 1: 42.843879781420817, -1: -41.257600000000004}, (1, -47): {0: 0.0, 1: 131.75358974358969, -1: -138.78749999999999}, (20, 7): {-1: -39.615430000000011}, (20, -35): {0: 0.0, 1: 128.01267909715392, -1: -112.90022988505748}, (5, -7): {0: 0.0, 1: 91.107329545454618, -1: -78.606086956521736}, (0, -48): {0: 0.0, 1: 128.74488679245283, -1: -114.58642857142856}, (10, 1): {0: 0.0, 1: 6.3234210526315771, -1: -7.9946153846153853}, (4, -8): {0: 0.0, 1: 58.617906976744237, -1: -64.914545454545447}, (15, -45): {0: 0.0, 1: 56.099217391304379, -1: -58.583846153846146}, (9, -35): {0: 0.0, 1: 29.91357377049183, -1: -29.848620689655171}, (3, -5): {0: 0.0, 1: 56.243544303797464, -1: -36.853333333333332}, (14, -42): {0: 0.0, 1: 40.085327102803717, -1: -44.233076923076922}, (10, 6): {0: 0.0, 1: 9.2873333333333328}, (0, -10): {0: 0.0, 1: 130.78112863479603, -1: -116.24765517241374}, (9, -48): {0: 0.0, 1: 29.206979166666638, -1: -19.530312500000001}, (2, -26): {0: 0.0, 1: 57.369105691056909, -1: -52.100000000000009}, (13, -19): {0: 0.0, 1: 27.692337662337646, -1: -23.217058823529413}, (15, 2): {-1: -43.560000000000002}, (15, -7): {0: 0.0, 1: 53.061627906976724, -1: -54.08937499999999}, (1, -19): {0: 0.0, 1: 133.45122302158273, -1: -95.539166666666674}, (12, -20): {0: 0.0, 1: 20.29832898172323, -1: -18.507288135593221}, (0, 13): {0: 0.0, -1: -17.624000000000009}, (14, -8): {0: 0.0, 1: 36.255463414634129, -1: -31.48043478260869}, (0, -20): {0: 0.0, 1: 129.63885379521167, -1: -109.69911564625846}, (11, -9): {0: 0.0, 1: 5.1592060730666054, -1: -5.303978723404259}, (1, 8): {1: 116.29000000000001}, (3, -13): {0: 0.0, 1: 55.311999999999998, -1: -73.278888888888872}, (15, -25): {0: 0.0, 1: 56.729568965517181, -1: -65.568749999999994}, (4, 3): {0: 0.0}, (10, -6): {0: 0.0, 1: 6.2004062179205075, -1: -7.2669868173258028}, (14, -30): {0: 0.0, 1: 33.670347826086953, -1: -23.744642857142857}, (5, 2): {0: 0.0, -1: -271.4076}, (9, -7): {0: 0.0, 1: 29.557749140893517, -1: -27.656222222222222}, (12, -2): {0: 0.0, 1: 19.249681762545865, -1: -15.726461538461541}, (3, -27): {0: 0.0, 1: 49.876707317073169, -1: -61.523333333333333}, (8, -8): {0: 0.0, 1: 66.033398437499898, -1: -63.497999999999998}, (7, 12): {1: 8.9550000000000125}, (16, -34): {0: 0.0, 1: 41.254596774193573, -1: -32.687777777777775}, (18, -18): {0: 0.0, 1: 57.86452380952381, -1: -43.015882352941183}, (20, -2): {0: 0.0, 1: 127.44373759647239, -1: -110.37852459016392}, (5, -40): {0: 0.0, 1: 94.595975609756124, -1: -46.620909090909095}, (7, -28): {0: 0.0, 1: 73.350046082949191, -1: -61.554000000000009}, (17, -11): {0: 0.0, 1: 67.794426229508176, -1: -50.078095238095251}, (19, -7): {0: 0.0, 1: 76.660465116279127, -1: -85.519500000000008}, (4, -39): {0: 0.0, 1: 57.083018867924523, -1: -59.289000000000001}, (8, -33): {0: 0.0, 1: 62.553832335329275, -1: -37.750740740740738}, (6, -19): {0: 0.0, 1: 66.530454545454575, -1: -82.186666666666667}, (16, -12): {0: 0.0, 1: 41.746446700507676, -1: -42.236875000000005}, (3, -38): {0: 48.906888888888879, 1: 57.282162162162152, -1: -56.049999999999997}, (20, 0): {0: 0.0, 1: 130.58346983951367, -1: -122.3197604790419}, (5, -30): {0: 0.0, 1: 79.792857142857159, -1: -84.497199999999992}, (0, -41): {0: 0.0, 1: 129.78696883852703, -1: -126.0520895522388}, (19, -33): {0: 0.0, 1: 80.979320388349464, -1: -69.795454545454533}, (4, -25): {0: 0.0, 1: 57.602016806722723, -1: -48.908235294117645}, (7, -50): {0: 3426.2307692307691, 1: 2774.6666666666665, -1: 3146.4651162790697}, (15, -40): {0: 0.0, 1: 52.400075187969982, -1: -48.295384615384606}, (11, 5): {0: 0.0}, (3, -32): {0: 0.0, 1: 49.005781250000005, -1: -48.166666666666664}, (19, -2): {0: 0.0, 1: 74.373000000000019, -1: -64.794242424242427}, (14, -39): {0: 0.0, 1: 37.616574074074073, -1: -39.534285714285716}, (20, -43): {0: 0.0, 1: 126.29400658616917, -1: -118.36226666666667}, (2, -31): {0: 0.0, 1: 53.444719999999997, -1: -43.689999999999998}, (13, -26): {0: 0.0, 1: 24.265920398009968, -1: -29.6247619047619}, (13, 7): {-1: -37.8155}, (1, -26): {0: 0.0, 1: 136.54264900662261, -1: -127.11714285714287}, (12, -21): {0: 0.0, 1: 18.639287500000009, -1: -17.374181818181814}, (0, 4): {1: 106.41333333333334}, (0, -29): {0: 0.0, 1: 135.19348979591857, -1: -125.47499999999994}, (20, 9): {1: 149.297}, (11, -20): {0: 0.0, 1: 5.9178779022568682, -1: -4.1364770240700208}, (1, 1): {0: 121.39397872340419, 1: 151.54846153846154, -1: -93.796666666666681}, (8, -46): {0: 0.0, 1: 68.179416058394168, -1: -30.534444444444446}, (15, -20): {0: 0.0, 1: 57.680638297872314, -1: -57.041249999999991}, (18, -41): {0: 0.0, 1: 63.100183486238556, -1: -51.10857142857143}, (10, -27): {0: 0.0, 1: 6.1482941498086161, -1: -6.1065227817745837}, (14, -27): {0: 0.0, 1: 36.226029411764728, -1: -29.041724137931034}, (5, 11): {-1: -168.34999999999999}, (9, -30): {0: 0.0, 1: 35.814093137254979, -1: -28.106666666666669}, (17, -40): {0: 0.0, 1: 66.556506024096365, -1: -88.38818181818182}, (3, -35): {0: 0.0, 1: 56.705957446808505, -1: -3.7725257731958592}, (13, -38): {0: 0.0, 1: 28.404083333333361, -1: -22.823636363636361}, (6, 0): {0: 0.0, 1: 65.983240740740769, -1: -49.892000000000003}, (17, -44): {0: 0.0, 1: 69.797999999999931, -1: -69.339047619047633}, (10, -49): {0: 0.0, 1: 6.6691013751199089, -1: -6.3247200000000046}, (12, -33): {0: 0.0, 1: 19.139940119760478, -1: -20.851470588235291}, (7, 5): {0: 0.0}, (16, -43): {0: 0.0, 1: 43.005161290322569, -1: -37.956666666666671}, (18, -23): {0: 0.0, 1: 57.769555555555534, -1: -60.517500000000005}, (11, -40): {0: 0.0, 1: 6.2446911624960553, -1: -7.8069019607843106}, (7, -35): {0: 0.0, 1: 63.15449367088604, -1: -61.407000000000004}, (17, -18): {0: 0.0, 1: 71.950746268656616, -1: -63.135106382978741}, (19, -1): {0: 0.0, 1: 77.220618556701083, -1: -68.909999999999997}, (7, -49): {0: 0.0, 1: 72.807183098591537, -1: -52.105000000000004}, (6, -28): {0: 0.0, 1: 68.383209876543233, -1: -69.585454545454553}, (16, -21): {0: 0.0, 1: 44.666516853932642, -1: -45.290769230769229}, (17, -36): {0: 0.0, 1: 68.40043478260867, -1: -76.160769230769219}, (20, -29): {0: 0.0, 1: 131.65633333333358, -1: -103.41016129032258}, (5, -21): {0: 0.0, 1: 95.10115853658543, -1: -85.303684210526299}, (7, -5): {0: 0.0, 1: 64.138694029850726, -1: -78.323846153846176}, (2, -34): {0: 0.0, 1: 56.886082474226818, -1: -52.141666666666673}, (19, -28): {0: 0.0, 1: 74.135522388059655, -1: -72.898666666666671}, (4, -18): {0: 0.0, 1: 61.282258064516192, -1: -53.643124999999998}, (6, -2): {0: 0.0, 1: 65.90565217391304, -1: -60.341111111111118}, (9, -37): {0: 0.0, 1: 31.328178694158058, -1: -28.287941176470582}, (3, -23): {0: 0.0, 1: 59.52154761904761, -1: -50.259999999999998}, (10, 9): {1: 43.584999999999994}, (2, -8): {0: 0.0, 1: 58.067076023391785, -1: -54.481250000000003}, (4, -12): {0: 0.0, 1: 56.923846153846213, -1: -54.998666666666658}, (15, -49): {0: 0.0, 1: 58.009383561643851, -1: -52.84571428571428}, (13, 0): {0: 0.0, 1: 31.466213991769553, -1: -21.301874999999999}, (5, -1): {0: 0.0, 1: 86.603081081081115, -1: -63.583636363636359}, (0, -6): {0: 0.0, 1: 133.33115139637482, -1: -128.71952941176474}, (11, -27): {0: 0.0, 1: 5.5255976621417977, -1: -4.6148241206030125}, (13, -7): {0: 0.0, 1: 28.159136363636357, -1: -23.893157894736845}, (15, -11): {0: 0.0, 1: 58.589249999999929, -1: -54.506999999999998}, (10, -20): {0: 0.0, 1: 6.7768312004874769, -1: -6.1874912891986025}, (2, 1): {0: 58.435200000000009, 1: 64.928000000000011, -1: -55.380000000000003}, (12, -8): {0: 0.0, 1: 18.610665859564161, -1: -14.367457627118647}, (14, -20): {0: 0.0, 1: 33.73492063492062, -1: -42.787894736842105}, (9, -21): {0: 0.0, 1: 32.8091013071896, -1: -36.483243243243251}, (18, -1): {0: 0.0, 1: 58.90147540983606, -1: -52.355294117647063}, (11, -5): {0: 0.0, 1: 5.7558796874999985, -1: -4.5180618556700995}, (13, -45): {0: 0.0, 1: 29.359203539823028, -1: -28.649117647058823}, (8, -26): {0: 0.0, 1: 52.745384615384609, -1: -83.664500000000004}, (10, -10): {0: 0.0, 1: 6.430224358974332, -1: -5.4824193548387159}, (12, -42): {0: 0.0, 1: 19.332075471698118, -1: -14.87833333333333}, (7, 2): {1: 91.958333333333329, -1: -54.276666666666664}, (19, -45): {0: 0.0, 1: 67.975172413793089, -1: -59.108124999999987}, (18, -32): {0: 0.0, 1: 57.707173913043469, -1: -54.555454545454538}, (11, -47): {0: 0.0, 1: 6.1217865779927569, -1: -4.1867460317460328}, (16, 10): {0: 66.025541560500002}, (7, -38): {0: 0.0, 1: 59.919642857142932, -1: -95.099999999999994}, (17, -25): {0: 0.0, 1: 68.28232558139527, -1: -58.708636363636373}, (10, -48): {0: 0.0, 1: 7.0610524737631026, -1: -5.9828294573643381}, (4, -49): {0: 0.0, 1: 62.035714285714278, -1: -60.238888888888887}, (17, 11): {1: 8.8560000000000088}, (6, -33): {0: 0.0, 1: 62.069649122807022, -1: -54.210000000000001}, (16, -30): {0: 0.0, 1: 36.365181818181803, -1: -39.43241379310345}, (18, -6): {0: 0.0, 1: 59.753087248322203, -1: -50.344166666666666}, (20, -22): {0: 0.0, 1: 137.33007125890791, -1: -132.94852173913048}, (5, -44): {0: 0.0, 1: 113.57884615384616, -1: -74.624285714285719}, (7, -16): {0: 0.0, 1: 69.955746268656597, -1: -79.725333333333339}, (17, -7): {0: 0.0, 1: 64.985521885521848, -1: -64.955111111111108}, (2, -39): {0: 0.0, 1: 61.275617977528043, -1: -59.979999999999997}, (19, -19): {0: 0.0, 1: 72.481967213114771, -1: -78.646842105263161}, (4, -43): {0: 0.0, 1: 58.920370370370357, -1: -54.401666666666664}, (8, 6): {1: 30.364000000000001}, (6, -15): {0: 0.0, 1: 61.309519230769247, -1: -53.305}, (16, -8): {0: 0.0, 1: 40.901345029239806, -1: -46.03923076923077}, (1, -34): {0: 0.0, 1: 124.84929411764705, -1: -107.44285714285715}, (3, -18): {0: 0.0, 1: 61.582222222222235, -1: -51.288301886792475}, (9, -47): {0: 0.0, 1: 30.709421768707475, -1: -18.728695652173911}, (5, -2): {0: 0.0, 1: 91.769940828402426, -1: -73.597999999999999}, (18, -2): {0: 0.0, 1: 57.151005917159743, -1: -51.363684210526323}, (0, -37): {0: 0.0, 1: 129.54700593723527, -1: -114.51893805309733}, (2, -1): {0: 0.0, 1: 59.175401459853973, -1: -54.25916666666668}, (2, -13): {0: 0.0, 1: 54.25681818181814, -1: -57.415199999999999}, (10, 4): {1: 43.5}, (4, -13): {0: 0.0, 1: 60.043274853801208, -1: -46.306249999999999}, (15, -44): {0: 0.0, 1: 58.706496350365008, -1: -52.140000000000001}, (1, -16): {0: 0.0, 1: 132.34613095238103, -1: -97.719333333333324}, (11, 9): {1: 59.4405}, (3, -12): {0: 0.0, 1: 54.925490196078428, -1: -52.571176470588227}, (14, 6): {1: 23.105}, (0, -15): {0: 0.0, 1: 129.47460513326789, -1: -124.6945695364238}, (2, -19): {0: 0.0, 1: 57.815096774193563, -1: -42.272307692307699}, (20, -24): {0: 0.0, 1: 135.97553658536626, -1: -110.73276923076925}, (13, -14): {0: 0.0, 1: 28.2479104477612, -1: -20.590833333333336}, (15, 7): {1: 170.88531714285713}, (15, -14): {0: 0.0, 1: 54.039110169491487, -1: -58.623076923076916}, (1, -22): {0: 0.0, 1: 119.86456647398859, -1: -116.60749999999997}, (12, -9): {0: 0.0, 1: 19.950928571428548, -1: -16.131688311688308}, (0, 8): {0: 0.0, -1: -9.8509999999999991}, (14, -9): {0: 0.0, 1: 35.091111111111104, -1: -31.433157894736837}, (9, -20): {0: 0.0, 1: 32.465581395348934, -1: -30.506249999999987}, (3, 5): {1: 129.27000000000001}, (11, -16): {0: 0.0, 1: 5.8702458226221141, -1: -5.8260767590618334}, (20, -46): {0: 0.0, 1: 129.63492553191486, -1: -118.72196721311477}, (8, -19): {0: 0.0, 1: 65.390955631399265, -1: -37.021818181818183}, (18, -39): {0: 0.0, 1: 60.143398058252394, -1: -58.74666666666667}, (10, -15): {0: 0.0, 1: 6.6210884452862979, -1: -6.4298000000000011}, (1, -27): {0: 0.0, 1: 120.90257352941163, -1: -113.90714285714286}, (5, 7): {0: 0.0}, (9, -10): {0: 0.0, 1: 28.98255813953492, -1: -20.459726027397263}, (11, -42): {0: 0.0, 1: 5.94750236220474, -1: -5.6038135593220302}, (16, 1): {0: 0.0, 1: 30.254444444444452, -1: -5.1100000000000003}, (7, -45): {0: 0.0, 1: 66.090720720720796, -1: -87.867500000000007}, (8, -13): {0: 0.0, 1: 55.869196141479037, -1: -42.690666666666658}, (10, -37): {0: 0.0, 1: 6.4447186384847583, -1: -4.7850364963503642}, (17, 12): {-1: -20.144000000000005}, (6, -42): {0: 0.0, 1: 63.785208333333337, -1: -66.652222222222221}, (16, -39): {0: 0.0, 1: 42.027499999999989, -1: -29.967000000000002}, (18, -11): {0: 0.0, 1: 54.878626373626339, -1: -59.574375000000011}, (18, 11): {0: 80.192688664005004}, (10, -47): {0: 0.0, 1: 6.1949892605093595, -1: -5.1188808664259877}, (7, -23): {0: 0.0, 1: 66.951007751937908, -1: -82.194999999999979}, (17, -14): {0: 0.0, 1: 70.668966666666535, -1: -45.667941176470592}, (18, -12): {0: 0.0, 1: 58.917499999999983, -1: -65.635999999999996}, (19, -14): {0: 0.0, 1: 75.123555555555512, -1: -59.767916666666657}, (4, -36): {0: 0.0, 1: 56.832117647058837, -1: -75.716666666666669}, (6, -24): {0: 0.0, 1: 60.724242424242433, -1: -50.98749999999999}, (1, -41): {0: 0.0, 1: 142.6914942528735, -1: -110.11714285714285}, (17, -34): {0: 0.0, 1: 59.764708994708961, -1: -63.074210526315795}, (9, 8): {-1: -14.995000000000001}, (5, -25): {0: 0.0, 1: 93.494814814814887, -1: -80.29249999999999}, (0, -46): {0: 0.0, 1: 127.9077425373135, -1: -103.76383720930234}, (19, -40): {0: 0.0, 1: 82.923580246913545, -1: -80.795217391304362}, (4, -6): {0: 0.0, 1: 56.918189655172455, -1: -51.652142857142856}, (15, -35): {0: 0.0, 1: 50.653868613138698, -1: -31.610000000000003}, (3, -3): {0: 0.0, 1: 58.232777777777805, -1: -69.53428571428573}, (14, -44): {0: 0.0, 1: 31.643152173913041, -1: -24.66076923076923}, (14, 9): {1: 49.5453245}, (12, 9): {0: 44.590792049999997}, (2, -28): {0: 0.0, 1: 52.601119999999995, -1: -43.461999999999996}, (13, -21): {0: 0.0, 1: 28.491767676767672, -1: -25.710769230769237}, (15, -5): {0: 0.0, 1: 57.170666666666619, -1: -39.638666666666673}, (1, -29): {0: 0.0, 1: 134.82699999999994, -1: -131.05692307692308}, (12, -18): {0: 0.0, 1: 20.498921933085462, -1: -22.838695652173907}, (0, 15): {0: 0.0}, (14, -2): {0: 0.0, 1: 36.381142857142855, -1: -27.706250000000001}, (0, -18): {0: 0.0, 1: 128.91218888342368, -1: -108.93213197969544}, (3, 10): {0: 0.0}, (11, -23): {0: 0.0, 1: 6.0215909090909303, -1: -3.8910792951541846}, (15, -31): {0: 0.0, 1: 59.4161111111111, -1: -60.565416666666671}, (10, -8): {0: 0.0, 1: 6.7579179532421501, -1: -5.8413333333333295}, (2, 13): {1: 242.435024}, (14, -32): {0: 0.0, 1: 38.521214285714308, -1: -33.76428571428572}, (5, 0): {0: 0.0, 1: 98.137919075144509, -1: -106.66461538461537}, (18, -46): {0: 0.0, 1: 58.967272727272729, -1: -50.560000000000009}, (9, -34): {0: 0.0, 1: 31.991869158878487, -1: -21.492058823529405}, (13, -33): {0: 0.0, 1: 33.113759398496256, -1: -17.43461538461538}, (8, -6): {0: 0.0, 1: 64.71805825242717, -1: -59.000909090909104}, (10, -44): {0: 0.0, 1: 7.2354215456674291, -1: -7.4294941634241303}, (17, 5): {0: 0.0}, (16, -48): {0: 0.0, 1: 40.953956043956005, -1: -34.976000000000006}, (18, -20): {0: 0.0, 1: 56.838380281690128, -1: -56.517307692307675}, (20, -8): {0: 0.0, 1: 127.46426052332249, -1: -113.56158940397349}, (7, -26): {0: 0.0, 1: 69.415639999999954, -1: -60.860833333333325}, (17, -21): {0: 0.0, 1: 66.083788395904378, -1: -58.974285714285706}, (19, -5): {0: 0.0, 1: 72.398922155688638, -1: -57.243999999999993}, (4, -37): {0: 0.0, 1: 58.155444444444463, -1: -85.093999999999994}, (6, -29): {0: 0.0, 1: 58.844823529411784, -1: -39.718000000000004}, (16, -10): {0: 0.0, 1: 42.710552763819187, -1: -32.294090909090912}, (3, -36): {0: 0.0, 1: 56.494285714285731, -1: -13.824999999999999}, (20, -26): {0: 0.0, 1: 133.99960698689998, -1: -116.84230088495578}, (5, -32): {0: 0.0, 1: 86.928387096774173, -1: -76.025882352941181}, (7, -4): {0: 0.0, 1: 72.551181434599087, -1: -56.390476190476193}, (19, -31): {0: 0.0, 1: 71.933239436619715, -1: -93.309999999999988}, (4, -31): {0: 0.0, 1: 61.397800000000018, -1: -78.436250000000001}, (8, -37): {0: 0.0, 1: 61.004060606060584, -1: -76.407499999999999}, (15, -38): {0: 0.0, 1: 57.235267175572552, -1: -61.181874999999998}, (11, 7): {0: 0.0}, (3, -30): {0: 0.0, 1: 48.780999999999992, -1: -49.779090909090904}, (20, 4): {1: 56.697142857142858}, (14, -33): {0: 0.0, 1: 37.643712121212111, -1: -25.350526315789473}, (0, -49): {0: 0.0, 1: 126.77626139817627, -1: -102.42808823529413}, (12, 0): {0: 0.0, 1: 22.993517647058859, -1: -21.682571428571425}, (13, -28): {0: 0.0, 1: 30.246899999999986, -1: -24.929090909090906}, (13, 5): {0: 0.0}, (1, -28): {0: 0.0, 1: 133.34303030303025, -1: -146.00666666666666}, (12, -27): {0: 0.0, 1: 18.705877862595418, -1: -12.324576271186441}, (0, 6): {-1: -69.596666666666664}, (0, -27): {0: 0.0, 1: 132.49663183773842, -1: -114.7227678571428}, (11, -18): {0: 0.0, 1: 5.7508144941478267, -1: -5.7860706401765931}, (7, -43): {0: 0.0, 1: 86.773235294117626, -1: -91.870000000000005}, (13, -1): {0: 0.0, 1: 31.507605633802807, -1: -41.780000000000001}, (15, -18): {0: 0.0, 1: 55.286897810218925, -1: -54.858499999999992}, (10, -29): {0: 0.0, 1: 6.4672806507844252, -1: -6.1976081424936398}, (2, 4): {-1: -37.663333333333334}, (14, -21): {0: 0.0, 1: 37.353371428571421, -1: -29.24909090909091}, (9, -32): {0: 0.0, 1: 31.07317585301837, -1: -32.005749999999992}, (13, -40): {0: 0.0, 1: 26.627888888888876, -1: -25.025833333333338}, (8, -31): {0: 0.0, 1: 60.257445652173878, -1: -55.036428571428573}, (18, -37): {0: 0.0, 1: 61.368913043478265, -1: -46.595454545454544}, (12, -39): {0: 0.0, 1: 21.493286052009456, -1: -19.762564102564099}, (16, -41): {0: 0.0, 1: 38.880119047619019, -1: -19.360000000000003}, (18, -25): {0: 0.0, 1: 61.538195488721776, -1: -46.021428571428579}, (11, -38): {0: 0.0, 1: 6.2324017467248751, -1: -5.4693877551020398}, (5, -49): {0: 0.0, 1: 96.321764705882359, -1: -87.527272727272717}, (7, -33): {0: 0.0, 1: 72.395696969696914, -1: -64.805500000000009}, (17, -20): {0: 0.0, 1: 69.091631419939532, -1: -54.896923076923066}, (10, -41): {0: 0.0, 1: 7.0359661495063266, -1: -5.2869999999999981}, (20, -33): {0: 0.0, 1: 129.95572332730549, -1: -129.35756097560974}, (6, -38): {0: 0.0, 1: 60.577763157894758, -1: -49.289999999999992}, (16, -19): {0: 0.0, 1: 43.750370370370426, -1: -40.006875000000008}, (20, -19): {0: 0.0, 1: 132.17392545136934, -1: -110.01771812080531}, (5, -23): {0: 0.0, 1: 98.479402173913158, -1: -67.881666666666661}, (20, -44): {0: 0.0, 1: 126.54370569280341, -1: -109.38587500000003}, (7, -11): {0: 0.0, 1: 61.292765957446726, -1: -62.046500000000002}, (2, -36): {0: 0.0, 1: 54.699753086419747, -1: -35.833999999999996}, (19, -26): {0: 0.0, 1: 76.226800000000026, -1: -82.36090909090909}, (4, -24): {0: 0.0, 1: 59.741666666666738, -1: -51.286249999999995}, (8, 1): {0: 0.0}, (6, -4): {0: 0.0, 1: 67.098623853211009, -1: -53.763333333333343}, (1, -37): {0: 0.0, 1: 118.76071428571426, -1: -188.98249999999999}, (3, -21): {0: 0.0, 1: 47.234479166666659, -1: -47.907142857142851}, (5, -13): {0: 0.0, 1: 89.058214285714342, -1: -72.233333333333334}, (4, -44): {0: 0.0, 1: 61.181044776119379, -1: -101.13}, (2, -10): {0: 0.0, 1: 60.078607594936742, -1: -53.57058823529411}, (4, -10): {0: 0.0, 1: 58.50938596491234, -1: -45.9895}, (1, -3): {0: 0.0, 1: 131.68141891891901, -1: -111.58928571428574}, (3, -15): {0: 0.0, 1: 49.109577464788721, -1: -43.689411764705881}, (19, -48): {0: 0.0, 1: 78.528717948717954, -1: -58.255714285714291}, (0, -4): {0: 0.0, 1: 135.22609304652352, -1: -128.32249999999993}, (11, -25): {0: 0.0, 1: 5.6978483532392419, -1: -4.8441964285714221}, (13, -9): {0: 0.0, 1: 27.871190476190463, -1: -27.277692307692309}, (15, -9): {0: 0.0, 1: 56.253426294820677, -1: -39.58428571428572}, (10, -22): {0: 0.0, 1: 6.4001270529903929, -1: -6.5832749562171644}, (12, -6): {0: 0.0, 1: 19.773116564417172, -1: -14.153035714285712}, (14, -14): {0: 0.0, 1: 36.526745283018862, -1: -27.854166666666668}, (9, -23): {0: 0.0, 1: 33.875158286778444, -1: -24.401521739130434}, (11, -3): {0: 0.0, 1: 5.6436365066876428, -1: -5.7358110882956925}, (7, -36): {0: 0.0, 1: 69.291543624161037, -1: -56.576666666666661}, (8, -24): {0: 0.0, 1: 61.807839999999949, -1: -52.336818181818188}, (10, -12): {0: 0.0, 1: 6.4206669644186176, -1: -5.6458687258687252}, (12, -48): {0: 0.0, 1: 22.364572127139343, -1: -16.870689655172416}, (16, -50): {0: 2297.6442307692309, 1: 1275.25, -1: 2426.3857142857141}, (9, -13): {0: 0.0, 1: 30.195633270321363, -1: -27.327121212121209}, (11, -45): {0: 0.0, 1: 5.7152184691664081, -1: -6.0135632183908063}, (7, -44): {0: 0.0, 1: 70.692966101694935, -1: -45.1447619047619}, (17, -27): {0: 0.0, 1: 66.506445312499935, -1: -48.699999999999989}, (10, -34): {0: 0.0, 1: 6.5834525894420581, -1: -5.7046451612903244}, (11, -1): {0: 0.0, 1: 5.6236586126266488, -1: -4.1207499999999975}, (6, -35): {0: 0.0, 1: 59.802295081967216, -1: -58.197999999999993}, (16, -28): {0: 0.0, 1: 36.630319999999983, -1: -35.640000000000001}, (18, -8): {0: 0.0, 1: 59.389583333333306, -1: -41.052857142857142}, (20, -12): {0: 0.0, 1: 125.927688470067, -1: -122.12512820512818}, (5, -46): {0: 0.0, 1: 77.925285714285721, -1: -59.487692307692313}, (7, -14): {0: 0.0, 1: 64.273441295546505, -1: -66.359166666666681}, (8, -44): {0: 0.0, 1: 67.38357664233574, -1: -75.838666666666668}, (2, -41): {0: 0.0, 1: 62.403428571428556, -1: -59.63600000000001}, (19, -17): {0: 0.0, 1: 79.385086705202355, -1: -56.935999999999993}, (4, -41): {0: 55.056103448275863, 1: 61.173448275862071, -1: -47.426428571428573}, (8, 8): {1: 56.986131699999994}, (6, -9): {0: 0.0, 1: 67.860096153846158, -1: -55.915999999999997}, (16, -6): {0: 0.0, 1: 41.940346534653493, -1: -41.018000000000001}, (1, -36): {0: 0.0, 1: 135.00987341772142, -1: -129.61958333333334}, (19, -42): {0: 0.0, 1: 72.562527472527492, -1: -62.903888888888872}, (9, 5): {-1: -48.271428571428579}, (5, -4): {0: 0.0, 1: 89.220752688172055, -1: -87.623636363636365}, (0, -35): {0: 0.0, 1: 131.18136963696369, -1: -105.33133928571421}, (2, -15): {0: 0.0, 1: 52.579885714285723, -1: -46.947333333333333}, (10, 2): {0: 0.0, 1: 14.430491803278692, -1: -18.513333333333332}, (4, -3): {0: 0.0, 1: 53.595555555555606, -1: -51.718461538461526}, (8, -39): {0: 0.0, 1: 67.656857142857106, -1: -64.75866666666667}, (15, -42): {0: 0.0, 1: 55.156556291390785, -1: -62.496428571428567}, (9, -29): {0: 0.0, 1: 32.69277272727274, -1: -20.530769230769231}, (1, -10): {0: 0.0, 1: 127.19385057471277, -1: -105.85555555555555}, (11, 11): {0: 7.9704000000000077}, (3, -10): {0: 0.0, 1: 54.210202020202026, -1: -61.931999999999995}, (2, -48): {0: 0.0, 1: 59.062089552238803, -1: -64.600000000000009}, (14, -45): {0: 0.0, 1: 36.055887850467286, -1: -50.465555555555561}, (0, -13): {0: 0.0, 1: 130.74082398829884, -1: -121.38362694300521}, (12, 12): {-1: -141.80000000000001}, (2, -21): {0: 0.0, 1: 58.086951219512216, -1: -50.800624999999997}, (13, -16): {0: 0.0, 1: 29.573382352941174, -1: -19.444482758620691}, (15, 1): {0: 0.0, 1: 47.301249999999996, -1: -30.739166666666666}, (15, -4): {0: 0.0, 1: 56.601060606060564, -1: -53.77473684210527}, (1, -24): {0: 0.0, 1: 142.00023391812886, -1: -133.385625}, (12, -15): {0: 0.0, 1: 20.485775656324599, -1: -13.756923076923076}, (14, -11): {0: 0.0, 1: 35.612692307692292, -1: -27.860624999999999}, (0, -23): {0: 0.0, 1: 128.98399564507366, -1: -108.70962264150937}, (11, -14): {0: 0.0, 1: 5.8763812858052251, -1: -5.142593320235763}, (13, -37): {0: 0.0, 1: 24.500225563909801, -1: -19.207777777777778}, (8, -17): {0: 0.0, 1: 55.761790540540446, -1: -55.665833333333332}, (4, 0): {0: 0.0, 1: 57.468670886076005, -1: -50.267333333333333}, (12, -49): {0: 0.0, 1: 22.004791666666659, -1: -21.187272727272727}, (5, 5): {-1: 1.1875999999999998}, (9, -12): {0: 0.0, 1: 31.740959860383949, -1: -21.139811320754717}, (8, -2): {0: 0.0, 1: 57.384935064935007, -1: -60.765769230769223}, (16, 3): {0: 82.046250000000001, 1: 61.956666666666671}, (6, 10): {0: 0.0}, (8, -11): {0: 0.0, 1: 61.086931034482667, -1: -41.139999999999993}, (9, -16): {0: 0.0, 1: 28.909892665474043, -1: -20.050487804878045}, (10, -39): {0: 0.0, 1: 6.5663190705582251, -1: -6.1560617760617689}, (17, 2): {0: 0.0, 1: 44.723333333333336, -1: -74.392499999999998}, (6, -44): {0: 0.0, 1: 66.308571428571426, -1: -68.217500000000001}, (16, -37): {0: 0.0, 1: 36.572077922077909, -1: -19.876923076923081}, (18, -13): {0: 0.0, 1: 56.276024844720475, -1: -45.261333333333347}, (20, -13): {0: 0.0, 1: 127.11354803493506, -1: -102.4453658536585}, (5, -37): {0: 0.0, 1: 96.830744680851012, -1: -80.693333333333328}, (7, -21): {0: 0.0, 1: 61.658772727272641, -1: -51.255625000000002}, (17, -16): {0: 0.0, 1: 60.890593749999979, -1: -64.679000000000016}, (19, -12): {0: 0.0, 1: 78.52476510067109, -1: -104.66499999999999}, (4, -34): {0: 0.0, 1: 56.276300000000028, -1: -69.243750000000006}, (6, -18): {0: 0.0, 1: 64.285416666666677, -1: -54.938000000000002}, (16, -15): {0: 0.0, 1: 41.474022346368763, -1: -40.705000000000013}, (1, -43): {0: 0.0, 1: 131.26530303030304, -1: -74.995555555555555}, (3, -39): {1: 60.688181818181825, -1: -33.669999999999995}, (20, -39): {0: 0.0, 1: 126.76659977703467, -1: -110.19999999999999}, (5, -27): {0: 0.0, 1: 88.476984126984121, -1: -71.039166666666659}, (0, -44): {0: 0.0, 1: 128.96719090909093, -1: -114.03858823529413}, (19, -38): {0: 0.0, 1: 70.387125000000026, -1: -89.449166666666656}, (16, -44): {0: 0.0, 1: 47.480760869565209, -1: -24.572222222222219}, (15, -33): {0: 0.0, 1: 51.608055555555588, -1: -42.523999999999994}, (1, -49): {0: 0.0, 1: 142.1295522388059, -1: -102.84333333333332}, (11, 0): {0: 0.0, 1: 5.6964627536007395, -1: -4.6095173745173721}, (17, -38): {0: 0.0, 1: 69.413999999999973, -1: -52.891666666666673}, (14, -38): {0: 0.0, 1: 39.620357142857166, -1: -36.133749999999999}, (2, -30): {0: 0.0, 1: 57.804356435643541, -1: -45.617000000000004}, (13, -23): {0: 0.0, 1: 26.45552511415524, -1: -24.992307692307691}, (1, -31): {0: 0.0, 1: 133.70799999999994, -1: -105.73333333333332}, (12, -24): {0: 0.0, 1: 19.369116517285541, -1: -14.601346153846158}, (0, 1): {0: 0.0, 1: 134.8821985815602, -1: -103.87846153846156}, (14, -4): {0: 0.0, 1: 36.280957446808515, -1: -27.075000000000003}, (0, -32): {0: 0.0, 1: 130.94479545454536, -1: -119.84315315315307}, (8, -34): {0: 0.0, 1: 54.862413793103457, -1: -49.380625000000002}, (11, -21): {0: 0.0, 1: 5.7746445105820206, -1: -6.0743227091633454}, (17, -49): {0: 0.0, 1: 69.051483516483501, -1: -68.379166666666677}, (15, -29): {0: 0.0, 1: 52.123623853210972, -1: -56.520454545454548}, (4, 15): {1: 242.2235}, (10, -26): {0: 0.0, 1: 6.6171566597653468, -1: -5.3445147679324938}, (10, -45): {0: 0.0, 1: 6.7193744441150285, -1: -4.8286397058823569}, (14, -26): {0: 0.0, 1: 37.536906077348071, -1: -29.414736842105263}, (20, -15): {0: 0.0, 1: 129.02555744199259, -1: -104.74528571428566}, (9, -3): {0: 0.0, 1: 27.604495268138795, -1: -22.072394366197187}, (13, -35): {0: 0.0, 1: 32.500096153846172, -1: -35.918499999999995}, (8, -4): {0: 0.0, 1: 52.565148514851415, -1: -31.709999999999997}, (12, -36): {0: 0.0, 1: 18.442285067873293, -1: -23.730909090909087}, (6, -49): {0: 0.0, 1: 62.469999999999985, -1: -51.324285714285715}, (18, -7): {0: 0.0, 1: 59.865310344827606, -1: -38.75555555555556}, (18, -22): {0: 0.0, 1: 59.377594936708846, -1: -56.655454545454539}, (8, -16): {0: 0.0, 1: 64.436258064516053, -1: -50.188800000000001}, (20, -6): {0: 0.0, 1: 125.51846153846216, -1: -117.81312138728323}, (7, -32): {0: 0.0, 1: 72.202678571428535, -1: -94.731000000000009}, (17, -23): {0: 0.0, 1: 70.603175675675587, -1: -58.806521739130432}, (19, -3): {0: 0.0, 1: 73.291871657754044, -1: -62.065882352941173}, (6, -31): {0: 0.0, 1: 64.33243902439024, -1: -71.682777777777758}, (16, -24): {0: 0.0, 1: 42.292468354430419, -1: -36.777000000000001}, (3, -34): {0: 0.0, 1: 58.927435897435899, -1: -59.902857142857144}, (20, -32): {0: 0.0, 1: 129.67214655172421, -1: -121.0425882352941}, (5, -18): {0: 0.0, 1: 88.907385620915051, -1: -43.329285714285717}, (7, -1): {0: 0.0, 1: 73.046811594202893, -1: -55.647600000000011}, (19, -29): {0: 0.0, 1: 74.31879629629627, -1: -105.06899999999999}, (4, -29): {0: 0.0, 1: 60.536379310344877, -1: -46.668461538461536}, (6, -5): {0: 0.0, 1: 62.289811320754708, -1: -74.665714285714287}, (3, -28): {0: 0.0, 1: 55.957368421052657, -1: -58.094444444444434}, (20, -50): {0: 6499.5074626865671, 1: 6757.4936936936938, -1: 6836.5893854748601}, (14, -35): {0: 0.0, 1: 37.694921874999991, -1: -44.244999999999997}, (12, 2): {-1: -14.523}, (2, -3): {0: 0.0, 1: 56.961986754966865, -1: -47.243157894736839}, (13, -30): {0: 0.0, 1: 28.770791366906497, -1: -14.124545454545455}, (8, -41): {0: 0.0, 1: 70.519465648854975, -1: -91.540909090909082}, (13, 3): {0: 0.0, 1: 6.3600000000000003}, (1, -6): {0: 0.0, 1: 128.75924050632923, -1: -99.817741935483866}, (1, -1): {0: 0.0, 1: 126.52602339181296, -1: -91.596875000000011}, (12, -25): {0: 0.0, 1: 18.394806408544706, -1: -13.909016393442624}, (0, -25): {0: 0.0, 1: 131.13326709526612, -1: -122.4111510791367}, (11, -32): {0: 0.0, 1: 5.3164466524722185, -1: -4.6802229299363063}, (1, 5): {1: 52.949999999999996}, (13, -4): {0: 0.0, 1: 30.001313131313147, -1: -19.644545454545451}, (15, -24): {0: 0.0, 1: 57.792348178137594, -1: -52.903684210526322}, (10, -31): {0: 0.0, 1: 6.6981786643538719, -1: -5.9657742782152248}, (2, 2): {1: 97.355200000000011}, (12, -3): {0: 0.0, 1: 19.953743961352661, -1: -16.790545454545452}, (14, -23): {0: 0.0, 1: 37.270588235294127, -1: -40.460625000000007}, (9, -26): {0: 0.0, 1: 31.965240963855397, -1: -27.72372093023256}, (13, -42): {0: 0.0, 1: 33.112275862068998, -1: -27.318235294117645}, (6, 4): {0: -50.823}, (8, -29): {0: 0.0, 1: 60.513939393939367, -1: -62.321034482758627}, (3, -48): {0: 0.0, 1: 47.452307692307677, -1: -32.512499999999996}, (12, -37): {0: 0.0, 1: 18.149135514018695, -1: -17.827200000000001}, (7, 1): {1: 62.537096774193557, -1: -46.236666666666672}, (18, -27): {0: 0.0, 1: 60.441407407407368, -1: -51.27210526315789}, (17, -41): {0: 0.0, 1: 62.28206451612899, -1: -43.871999999999986}, (11, -36): {0: 0.0, 1: 5.5730032229709732, -1: -5.4163709677419352}, (18, -50): {0: 2579.8333333333335, 1: 2786.532786885246, -1: 3059.5799999999999}, (7, -39): {0: 0.0, 1: 62.711275167785246, -1: -72.250555555555536}, (17, -30): {0: 0.0, 1: 68.038308270676595, -1: -63.188888888888904}, (18, -33): {0: 0.0, 1: 55.190535714285723, -1: -59.790588235294109}, (19, 2): {1: 128.10814583333331, -1: -84.815000000000012}, (9, -45): {0: 0.0, 1: 31.845673758865267, -1: -27.663461538461544}, (6, -40): {0: 0.0, 1: 56.371052631578934, -1: -68.177777777777763}, (16, -17): {0: 0.0, 1: 39.330206185567064, -1: -35.585833333333341}, (20, 5): {1: 97.9707619047619}, (20, -17): {0: 0.0, 1: 129.51362302483136, -1: -126.21709459459456}, (5, -41): {0: 0.0, 1: 105.99499999999995, -1: -98.155384615384619}, (7, -9): {0: 0.0, 1: 73.080862068965416, -1: -70.24045454545454}, (2, -38): {0: 0.0, 1: 54.340987654320976, -1: -51.625}, (19, -24): {0: 0.0, 1: 75.286197183098594, -1: -53.123333333333328}, (4, -22): {0: 0.0, 1: 54.283125000000013, -1: -56.155833333333341}, (8, 3): {-1: -60.165714285714294}, (6, -14): {0: 0.0, 1: 59.318533333333335, -1: -49.335714285714289}, (18, -15): {0: 0.0, 1: 59.600597826086975, -1: -57.579999999999991}, (3, -19): {0: 0.0, 1: 54.682898550724651, -1: -55.911111111111126}, (9, 2): {-1: -23.36375}, (5, -15): {0: 0.0, 1: 85.730606060606121, -1: -90.156500000000008}, (14, -1): {0: 0.0, 1: 37.009499999999967, -1: -42.767894736842109}, (2, -12): {0: 0.0, 1: 55.168901734104033, -1: -46.163333333333334}, (19, -50): {0: 4528.4913793103451, 1: 2874.3333333333335, -1: 4136.0625}, (4, -16): {0: 0.0, 1: 57.699609375000058, -1: -36.826666666666668}, (1, -13): {0: 0.0, 1: 134.78193717277506, -1: -129.70879999999997}, (8, -49): {0: 0.0, 1: 66.792272727272731, -1: -53.177272727272722}, (14, -50): {0: 2150.934065934066, 1: 1573.5833333333333, -1: 1616.0}, (14, 3): {1: 55.016666666666673}, (0, -2): {0: 0.0, 1: 134.01894190871397, -1: -121.58703703703701}, (2, -18): {0: 0.0, 1: 54.437814569536393, -1: -38.121249999999996}, (13, -11): {0: 0.0, 1: 31.293333333333333, -1: -17.581578947368424}, (15, -15): {0: 0.0, 1: 54.213333333333289, -1: -53.876666666666672}, (10, -24): {0: 0.0, 1: 7.0203500712814906, -1: -4.6877892561983474}, (19, 0): {0: 0.0, 1: 74.17791666666669, -1: -66.793243243243253}, (12, -12): {0: 0.0, 1: 18.924719101123568, -1: -22.829000000000008}, (14, -16): {0: 0.0, 1: 36.380955056179751, -1: -20.916315789473689}, (9, -17): {0: 0.0, 1: 32.332850393700845, -1: -27.712702702702693}, (3, 0): {0: 0.0, 1: 57.046086956521748, -1: -61.744000000000007}, (13, -49): {0: 0.0, 1: 30.539274193548408, -1: -36.661250000000003}, (8, -22): {0: 0.0, 1: 64.601362007168333, -1: -59.606956521739143}, (10, -14): {0: 0.0, 1: 6.3997647750661466, -1: -6.4542094455852173}, (12, -46): {0: 0.0, 1: 20.863704600484237, -1: -11.728157894736842}, (8, -50): {0: 4033.2222222222222, 1: 3948.875, -1: 4028.5277777777778}, (9, -15): {0: 0.0, 1: 28.753634686346857, -1: -32.175714285714285}, (11, -43): {0: 0.0, 1: 5.8474718045112652, -1: -4.1490322580645156}, (7, -42): {0: 0.0, 1: 69.034555555555599, -1: -51.898124999999993}, (17, -37): {0: 0.0, 1: 68.774294871794851, -1: -60.671578947368424}, (19, 11): {0: 0.0}, (6, -45): {0: 0.0, 1: 59.008833333333321, -1: -69.450000000000003}, (16, -26): {0: 0.0, 1: 40.846176470588254, -1: -52.384999999999991}, (20, 12): {1: 157.15000000000001, -1: -22.299000000000007}, (20, -10): {0: 0.0, 1: 126.87734794520618, -1: -108.12078571428574}, (5, -48): {0: 0.0, 1: 78.822613636363641, -1: -75.705454545454543}, (7, -20): {0: 0.0, 1: 63.51932203389817, -1: -66.920434782608694}, (17, -3): {0: 0.0, 1: 63.329182156133747, -1: -53.191250000000011}, (2, -43): {0: 0.0, 1: 58.992564102564103, -1: -41.831111111111113}, (19, -15): {0: 0.0, 1: 73.761091954023016, -1: -44.276874999999997}, (4, -47): {0: 0.0, 1: 58.518510638297883, -1: -65.578000000000003}, (8, 10): {-1: -20.916639999999994}, (6, -11): {0: 0.0, 1: 65.662068965517236, -1: -50.705714285714279}, (16, -4): {0: 0.0, 1: 40.082580645161336, -1: -41.262777777777778}, (1, -46): {0: 0.0, 1: 135.39440476190475, -1: -168.58636363636364}, (3, -46): {0: 0.0, 1: 66.807878787878778, -1: -50.395714285714291}, (20, -36): {0: 0.0, 1: 128.75955974842768, -1: -116.53678571428571}, (5, -6): {0: 0.0, 1: 85.798773006135022, -1: -71.780000000000015}, (0, -33): {0: 0.0, 1: 130.88575449569998, -1: -120.26287128712868}, (2, -49): {0: 0.0, 1: 54.274390243902431, -1: -49.62166666666667}, (10, 0): {0: 0.0, 1: 6.3902492753623017, -1: -4.7786329588014977}, (15, -48): {0: 0.0, 1: 55.429065420560789, -1: -33.235882352941175}, (8, -36): {0: 0.0, 1: 63.099780219780172, -1: -66.150833333333324}, (1, -12): {0: 0.0, 1: 128.79084337349408, -1: -100.48090909090909}, (11, 13): {1: 148.34}, (3, -8): {0: 0.0, 1: 60.249999999999957, -1: -54.664999999999999}, (14, -47): {0: 0.0, 1: 37.785725190839678, -1: -40.644000000000005}, (14, 10): {1: 73.361712845}, (0, -11): {0: 0.0, 1: 131.59762686567197, -1: -115.45673611111107}, (2, -23): {0: 0.0, 1: 55.177259259259223, -1: -50.375}, (13, -18): {0: 0.0, 1: 28.628122065727712, -1: -26.124444444444443}, (17, -2): {0: 0.0, 1: 67.325851393188714, -1: -64.236153846153854}, (15, -1): {0: 0.0, 1: 55.783999999999949, -1: -43.488181818181815}, (1, -18): {0: 0.0, 1: 133.02167664670668, -1: -113.87642857142855}, (12, -13): {0: 0.0, 1: 18.002650602409634, -1: -17.694909090909089}, (14, -5): {0: 0.0, 1: 37.559374999999996, -1: -30.68933333333333}, (0, -21): {0: 0.0, 1: 130.11831849135712, -1: -122.66330985915486}, (3, 9): {0: -208.90413000000001}, (11, -12): {0: 0.0, 1: 5.593102695963875, -1: -4.3783040935672437}, (1, 9): {-1: -232.1157}, (11, -49): {0: 0.0, 1: 5.7924279973208206, -1: -4.761265822784809}, (15, -28): {0: 0.0, 1: 63.915813953488311, -1: -47.757368421052632}, (4, 2): {0: 0.0}, (10, -3): {0: 0.0, 1: 6.1780696296296416, -1: -5.6219397363465129}, (5, 3): {1: 150.10050000000001}, (9, -6): {0: 0.0, 1: 27.286260575296133, -1: -27.508793103448284}, (10, 8): {0: 0.0}, (16, 5): {0: -57.405000000000001}, (6, 8): {-1: -59.145000000000003}, (8, -9): {0: 0.0, 1: 60.847662835248961, -1: -58.816666666666656}, (17, 0): {0: 0.0, 1: 68.453175074183946, -1: -65.734571428571442}, (16, -35): {0: 0.0, 1: 37.62674999999998, -1: -34.410000000000004}, (7, -2): {0: 0.0, 1: 74.595259515570874, -1: -80.969565217391278}, (20, 11): {0: 0.0, -1: -140.97999999999999}, (17, -47): {0: 0.0, 1: 64.152688172043014, -1: -40.974615384615383}, (20, -3): {0: 0.0, 1: 128.24936343852045, -1: -114.69882352941174}, (5, -39): {0: 0.0, 1: 93.492346938775526, -1: -84.720769230769221}, (7, -27): {0: 0.0, 1: 62.249095022624346, -1: -62.422000000000011}, (17, -10): {0: 0.0, 1: 64.126042402826855, -1: -76.49666666666667}, (19, -10): {0: 0.0, 1: 79.145937499999974, -1: -76.702857142857141}, (4, -40): {0: 0.0, 1: 61.738939393939397, -1: -45.273333333333333}, (6, -20): {0: 0.0, 1: 61.6329347826087, -1: -51.404444444444437}, (16, -13): {0: 0.0, 1: 39.586193181818224, -1: -33.256666666666675}, (3, -41): {1: 51.15868421052631, -1: -23.59333333333333}, (3, -37): {0: 0.0, 1: 58.652105263157885, -1: -87.170000000000002}, (20, -37): {0: 0.0, 1: 127.49591208791207, -1: -125.00999999999999}, (5, -29): {0: 0.0, 1: 90.731171875000001, -1: -74.074615384615399}, (0, -42): {0: 0.0, 1: 130.98723222748816, -1: -114.93184210526316}, (9, -43): {0: 0.0, 1: 29.399133574007191, -1: -28.354583333333338}, (19, -36): {0: 0.0, 1: 71.88883720930238, -1: -44.89142857142857}, (4, -26): {0: 0.0, 1: 59.688250000000082, -1: -46.876153846153848}, (15, -39): {0: 0.0, 1: 53.023545454545463, -1: -62.942499999999995}, (11, 2): {1: 10.581599999999998, -1: -19.547619047619044}, (3, -31): {0: 0.0, 1: 60.614927536231889, -1: -51.696666666666658}, (14, -40): {0: 6.1999297705651895, 1: 35.544583333333328, -1: -49.948888888888881}, (12, 5): {0: 0.0, 1: -13.3695, -1: -5.3899999999999997}, (2, -32): {0: 0.0, 1: 53.729670329670334, -1: -53.702352941176471}, (13, -25): {0: 0.0, 1: 27.791835748792256, -1: -18.647222222222222}, (15, 8): {1: 76.129999999999995}, (3, -43): {0: 0.0, 1: 48.804999999999993, -1: -55.998333333333335}, (13, 8): {-1: 23.216450000000002}, (1, -25): {0: 0.0, 1: 124.4800625000001, -1: -101.66}, (12, -22): {0: 0.0, 1: 20.836780905752757, -1: -22.06428571428571}, (0, 3): {0: 0.0, 1: 198.96216666666669, -1: -93.953333333333333}, (9, 6): {-1: -2.1200000000000001}, (0, -30): {0: 0.0, 1: 135.57137377963755, -1: -127.98892561983465}, (11, -19): {0: 0.0, 1: 5.8182920912178746, -1: -3.7903999999999982}, (1, 2): {0: 82.762500000000003}, (15, -19): {0: 0.0, 1: 57.627602996254637, -1: -57.648695652173927}, (18, -9): {0: 0.0, 1: 56.457771428571455, -1: -43.702727272727266}, (10, -28): {0: 0.0, 1: 6.2824120413922584, -1: -7.2134669811320773}, (14, -28): {0: 0.0, 1: 34.999999999999979, -1: -36.412727272727281}, (18, 2): {0: 0.0}, (18, 7): {-1: -2.9924299999999988}, (4, -2): {0: 0.0, 1: 57.280780141844041, -1: -49.756666666666668}, (6, 3): {1: 85.551249999999996}, (8, -1): {0: 0.0, 1: 63.999088050314462, -1: -47.463478260869557}, (10, -50): {0: 315.18387276785717, 1: 350.18996415770607, -1: 255.85650224215246}, (12, -34): {0: 0.0, 1: 18.648308026030364, -1: -11.032941176470587}, (7, 10): {0: -118.773}, (6, -36): {0: 0.0, 1: 65.987307692307695, -1: -34.738}, (18, -24): {0: 0.0, 1: 61.153924050632888, -1: -50.820714285714288}, (11, -39): {0: 0.0, 1: 6.0531392089691529, -1: -5.5193939393939404}, (7, -30): {0: 0.0, 1: 65.677142857142755, -1: -64.427586206896564}, (17, -17): {0: 0.0, 1: 62.800955631399269, -1: -63.818800000000017}, (6, -25): {0: 0.0, 1: 65.032906976744201, -1: -83.200000000000003}, (16, -22): {0: 0.0, 1: 43.964406779661076, -1: -30.673749999999998}, (20, 1): {0: 0.0, 1: 127.13196319018402, -1: -134.821}, (5, -20): {0: 0.0, 1: 95.882000000000048, -1: -59.25888888888889}, (7, -8): {0: 0.0, 1: 70.811692307692169, -1: -50.899444444444441}, (20, -48): {0: 0.0, 1: 128.20509670079633, -1: -120.59701149425288}, (19, -27): {0: 0.0, 1: 72.228827586206904, -1: -83.501764705882351}, (4, -19): {0: 0.0, 1: 58.160633802816974, -1: -43.710000000000001}, (6, -7): {0: 0.0, 1: 67.89547008547008, -1: -68.971666666666664}, (3, -26): {0: 0.0, 1: 56.773900000000012, -1: -50.589999999999996}, (5, -10): {0: 0.0, 1: 91.605380952381026, -1: -111.97750000000001}, (8, -35): {0: 0.0, 1: 62.43533742331288, -1: -57.863749999999996}, (2, -5): {0: 0.0, 1: 58.188675496688703, -1: -48.092916666666667}, (13, -32): {0: 0.0, 1: 24.668235294117661, -1: -34.267777777777781}, (13, 1): {0: 0.0, 1: 34.592068965517235, -1: -47.82200000000001}, (1, -8): {0: 0.0, 1: 131.31559006211191, -1: -91.327777777777769}, (10, -46): {0: 0.0, 1: 7.0688108754108061, -1: -7.0601851851851878}, (12, -31): {0: 0.0, 1: 18.83855917667238, -1: -16.381568627450985}, (0, -7): {0: 0.0, 1: 132.94729206049209, -1: -122.56979999999999}, (17, -50): {0: 3334.8024193548385, 1: 1916.1666666666667, -1: 3066.0957446808511}, (11, -30): {0: 0.0, 1: 5.9153851174935017, -1: -4.2937127371273665}, (13, -6): {0: 0.0, 1: 30.555510204081632, -1: -32.94533333333333}, (8, -45): {0: 0.0, 1: 67.003225806451624, -1: -41.282000000000004}, (15, -22): {0: 0.0, 1: 55.613039647577054, -1: -48.414285714285718}, (18, -44): {0: 0.0, 1: 53.728061224489778, -1: -56.060000000000002}, (10, -17): {0: 0.0, 1: 6.401041666666659, -1: -4.7300754716981155}, (2, 0): {0: 0.0, 1: 53.873313253012043, -1: -49.583333333333336}, (18, -4): {0: 0.0, 1: 59.403282051282041, -1: -43.519999999999996}, (14, -17): {0: 0.0, 1: 35.37788990825689, -1: -35.785000000000004}, (9, -28): {0: 0.0, 1: 30.617175257731954, -1: -31.864418604651163}, (11, -8): {0: 0.0, 1: 5.5764389557605094, -1: -4.7894495412844078}, (13, -44): {0: 0.0, 1: 31.481691176470605, -1: -18.261000000000003}, (8, -27): {0: 0.0, 1: 59.560074074074009, -1: -55.643076923076919}}

In [38]:
def generate_mle_policy(Q, time, price):
    policy = []
    cur_num_coins = 0
    for t in range(time, time+hp.get_policy_length()):
        x = [t+i for i in range(tau)]
        y = price[t-tau:t]
        slope, intercept = np.polyfit(x, y, 1)
        interval_group = hp.get_interval_enum(slope[0])
        if (interval_group, cur_num_coins) in Q: 
            Q_value = Q[(interval_group, cur_num_coins)]
            action = max(Q_value, key=Q_value.get)
            if Q_value[action] < 0: # meaningless maximum
                action = random.choice([-1, 0, 1])
        else:
            action = random.choice([-1, 0, 1])
            
        if action == -1:
            cur_num_coins += 1
        elif action == 1:
            cur_num_coins -= 1
        else:
            pass
        
        policy.append(action)
    return policy

In [39]:
def run():
    reward_array = []
    test_set = hp.get_30_t_for_eval();
    for random_time in test_set:
        policy = generate_mle_policy(Q, random_time, price)
        reward = evaluate.evaluate_reward(policy, price, random_time)
        reward_array.append(reward)

    evaluate.print_statistics_95_confidence(reward_array)

In [40]:
for i in range(10):
    run()

('average:', array([-144.14466667]), 'standard_error', 154.4775385615728)
('average:', array([-143.777]), 'standard_error', 154.14100658415865)
('average:', array([-143.52633333]), 'standard_error', 153.91301482132357)
('average:', array([-141.22066667]), 'standard_error', 151.87193274140705)
('average:', array([-145.308]), 'standard_error', 155.55882238108401)
('average:', array([-142.43166667]), 'standard_error', 152.93126767041468)
('average:', array([-143.577]), 'standard_error', 153.95900327760518)
('average:', array([-144.69466667]), 'standard_error', 154.98564609858261)
('average:', array([-142.601]), 'standard_error', 153.08164466622969)
('average:', array([-142.59733333]), 'standard_error', 153.07838266813934)


In [18]:
print reward_array

NameError: name 'reward_array' is not defined